In [2]:
import os
import re
import h5py
import json

import numpy as np

from tqdm import tqdm

In [3]:
os.chdir('/home/fyhsieh/data_augmentation')

1. Sample preparation
* Generate .root file
* Select the events and save jet constitutent information to .h5 file
    - Sample/from_root_to_h5.py
* Generate mixed sample
    - split training and validation datasets
    - true label testing dataset
* Data augmentation
    - optional
    - apply on training dataset
    - Sample/physical_augmentation_h5.ipynb
* From .h5 file generate jet image and save in .npy file
    - Sample/from_h5_to_npy.py

2. CNN Training

# Sample preparation

## From root to h5

In [2]:
root_path = '../../Software/pythia8307/HVmodel/test_100k-1.root'
output_path = './HVmodel/data/split_val/signal.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = '../../Software/pythia8307/HVmodel/test_100k-2.root'
output_path = './HVmodel/data/split_val/signal-val.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

for i in range(3, 5):
    root_path = f'../../Software/pythia8307/HVmodel/test_100k-{i}.root'
    output_path = f'./HVmodel/data/split_val/signal_{i}.h5'
    sample_type = 1

    cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
    print(cmd)

root_path = '../../Software/pythia8307/HVmodel/test_100k-5.root'
output_path = './HVmodel/data/split_val/signal-test_2.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)


root_path = './ppjj/Events/run_03/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background_03.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_04/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background_04.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_05/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background-val.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

for i in range(6, 12):
    root_path = f'./ppjj/Events/run_{i:02}/tag_1_delphes_events.root'
    output_path = f'./HVmodel/data/split_val/background_{i:02}.h5'
    sample_type = 0

    cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
    print(cmd)

root_path = './ppjj/Events/run_12/tag_1_delphes_events.root'
output_path = './HVmodel/data/split_val/background-test_2.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-1.root ./HVmodel/data/split_val/signal.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-2.root ./HVmodel/data/split_val/signal-val.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-3.root ./HVmodel/data/split_val/signal_3.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-4.root ./HVmodel/data/split_val/signal_4.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-5.root ./HVmodel/data/split_val/signal-test_2.h5 1 &
python from_root_to_h5.py ./ppjj/Events/run_03/tag_1_delphes_events.root ./HVmodel/data/split_val/background_03.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_04/tag_1_delphes_events.root ./HVmodel/data/split_val/background_04.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_05/tag_1_delphes_events.root ./HVmodel/data/split_val/background-val.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_06/

## Generate mixed sample

In [2]:
def get_info(path):
    # path: run path
    name = os.path.split(path)[1]

    with open(os.path.join(path, f'{name}_tag_1_banner.txt')) as f:
        for line in f.readlines():
                
            #  Integrated weight (pb)  :       0.020257
            match = re.match('#  Integrated weight \(pb\)  : +(\d+\.\d+)', line)
            if match:
                # unit: fb
                cross_section = float(match.group(1)) * 1000
            # #  Number of Events        :       100000
            match = re.match('#  Number of Events        :       (\d+)', line)
            if match:
                # unit: fb
                nevent = int(match.group(1))
    
    return cross_section, nevent

def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

def create_dataset(f, nevent, MAX_JETS):

    f.create_dataset('J1/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('J2/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('EVENT/Mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/signal', (nevent,), maxshape=(None,), dtype='<i8')

### Pre-selection 

In [3]:
import ROOT

ROOT.gROOT.ProcessLine('.include /usr/local/Delphes-3.4.2/')
ROOT.gROOT.ProcessLine('.include /usr/local/Delphes-3.4.2/external/')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootTreeReader.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootConfReader.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootTask.h"')
ROOT.gSystem.Load("/usr/local/Delphes-3.4.2/install/lib/libDelphes")

Welcome to JupyROOT 6.20/08


0

In [4]:
def Mjets(*arg):
    # arg: list of jets
    # return: invariant mass of jets
    e_tot, px_tot, py_tot, pz_tot = 0, 0, 0, 0
    
    for jet in arg:
        pt, eta, phi, m = jet[0], jet[1], jet[2], jet[3]
        
        px, py, pz = pt*np.cos(phi), pt*np.sin(phi), pt*np.sinh(eta)
        e = np.sqrt(m**2 + px**2 + py**2 + pz**2)
        
        px_tot += px
        py_tot += py
        pz_tot += pz
        e_tot += e
    
    return np.sqrt(e_tot**2 - px_tot**2 - py_tot**2 - pz_tot**2)

In [5]:
def HV_selection(tree):
    # Hidden Valley model selection
    # 1. 2 jets
    # 2. pT > 750 GeV
    # 3. |eta| < 2.0

    SRSB_region = [4400, 4700, 5500, 5800]

    n_event_count = 0
    n_jet_count = 0
    jet_pt_count = 0
    jet_eta_count = 0
    mjjs = []

    pt = [[],[]]

    for event_id, event in tqdm(enumerate(tree)):
        n_event_count += 1

        if event.Jet_size < 2:
            continue
        n_jet_count += 1

        pt[0].append(event.Jet[0].PT)
        pt[1].append(event.Jet[1].PT)
        if event.Jet[1].PT < 750:
            continue
        jet_pt_count += 1

        if abs(event.Jet[0].Eta) > 2.0 or abs(event.Jet[1].Eta) > 2.0:
            continue
        jet_eta_count += 1

        jets = [[event.Jet[i].PT, event.Jet[i].Eta, event.Jet[i].Phi, event.Jet[i].Mass] for i in range(2)]
        mjj = Mjets(*jets)
        mjjs.append(mjj)

        if mjj < SRSB_region[0] or mjj > SRSB_region[3]:
            continue


    mjjs = np.array(mjjs)
    SR_count = ((mjjs > SRSB_region[1]) & (mjjs < SRSB_region[2])).sum()
    SB_count = (((mjjs > SRSB_region[0]) & (mjjs < SRSB_region[1])) | ((mjjs > SRSB_region[2]) & (mjjs < SRSB_region[3]))).sum()

    cutflow_number = {
        'Total': n_event_count,
        'n jet cut': n_jet_count,
        'jet pt cut': jet_pt_count,
        'jet eta cut': jet_eta_count,
        'Signal region': SR_count,
        'Sideband region': SB_count,
    }

    results = {
        'mjj': mjjs,
        'pt': np.array(pt),
        'cutflow_number': cutflow_number,
    }

    return results

In [7]:
root_file = '../Software/pythia8307/HVmodel/test_100k-1.root'
f = ROOT.TFile(root_file)
tree_s = f.Get("Delphes")

results_s = HV_selection(tree_s)

0it [00:00, ?it/s]

100000it [08:58, 185.74it/s]


In [8]:
root_file = './Sample/ppjj/Events/run_02/tag_1_delphes_events.root'
f = ROOT.TFile(root_file)
tree_b = f.Get("Delphes")

results_b = HV_selection(tree_b)

100000it [07:45, 215.05it/s]
Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo for version 2 of class GenParticle read from the file ./Sample/ppjj/Events/run_02/tag_1_delphes_events.root
   has a different checksum than the previously loaded StreamerInfo.
   Reading objects of type GenParticle from the file ./Sample/ppjj/Events/run_02/tag_1_delphes_events.root 
   (and potentially other files) might not work correctly.
   Most likely the version number of the class was not properly
   updated [See ClassDef(GenParticle,2)].
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float T; //number
vs
   float CtgTheta; //number
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float X; //number
vs
   float D0; //number
Warning in <TStreame

In [9]:
# modified SB region to 4400-4700, 5500-5800
np.save('./Sample/HVmodel/data/selection_results_SB_4400_5800_s.npy', results_s)
np.save('./Sample/HVmodel/data/selection_results_SB_4400_5800_b.npy', results_b)

### Compute sample size

In [3]:
results_s = np.load('./Sample/HVmodel/data/selection_results_SB_4400_5800_s.npy', allow_pickle=True).item()
results_b = np.load('./Sample/HVmodel/data/selection_results_SB_4400_5800_b.npy', allow_pickle=True).item()

In [4]:
# Total cross section and number of events
xection, tot_event = get_info('./Sample/ppjj/Events/run_03')
print(xection, tot_event)

# cross section in signal region and sideband region
cross_section_SR = results_b['cutflow_number']['Signal region'] / results_b['cutflow_number']['Total'] * xection
cross_section_SB = results_b['cutflow_number']['Sideband region'] / results_b['cutflow_number']['Total'] * xection
print(f'Background cross section, SR: {cross_section_SR:.2f} fb, SB: {cross_section_SB:.2f} fb')

# number of background events in signal region and sideband region
L = 139 * 1
n_SR_B = cross_section_SR * L
n_SB_B = cross_section_SB * L

print(f'Background sample size: SR: {n_SR_B:.1f}, SB: {n_SB_B:.1f}')

sensitivity = 1
n_SR_S = sensitivity * np.sqrt(n_SR_B)
n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']
print(f'Signal sample size: SR: {n_SR_S:.1f}, SB: {n_SB_S:.1f}')

6837.392481 1000000
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 137.6, SB: 33.5


In [5]:
# Training and validation splitting ratio
r_train = 0.8
r_val = 0.2

### Make Training and Validation datasets

In [4]:
def create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, SRSB_region=[4400, 4700, 5500, 5800], seed=0):
    # n_sig_1: number of signal events in mixing sample 1 (Signal region)
    # n_sig_2: number of signal events in mixing sample 2 (Sideband region)

    nevent = n_sig_1 + n_sig_2 + n_bkg_1 + n_bkg_2
    with h5py.File(output_path, 'w') as f_out:
        MAX_JETS = 300 
        create_dataset(f_out, nevent, MAX_JETS)

        keys = get_dataset_keys(f_out)
        with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg: 

            mjj_s = f_sig['EVENT/Mjj'][:]
            mjj_b = f_bkg['EVENT/Mjj'][:]
            SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
            SB_range_s = ((mjj_s > SRSB_region[0]) & (mjj_s < SRSB_region[1])) | ((mjj_s > SRSB_region[2]) & (mjj_s < SRSB_region[3]))
            SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])
            SB_range_b = ((mjj_b > SRSB_region[0]) & (mjj_b < SRSB_region[1])) | ((mjj_b > SRSB_region[2]) & (mjj_b < SRSB_region[3]))

            # randomly pick events
            np.random.seed(seed)
            idx_SR_s = np.random.choice(np.where(SR_range_s)[0], n_sig_1, replace=False)
            idx_SB_s = np.random.choice(np.where(SB_range_s)[0], n_sig_2, replace=False)
            idx_SR_b = np.random.choice(np.where(SR_range_b)[0], n_bkg_1, replace=False)
            idx_SB_b = np.random.choice(np.where(SB_range_b)[0], n_bkg_2, replace=False)

            for key in keys:
                # f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
                # f_out[key][n_sig_1:n_sig_1+n_bkg_1] = f_bkg[key][:][SR_range_b][:n_bkg_1]
                # f_out[key][n_sig_1+n_bkg_1:n_sig_1+n_bkg_1+n_sig_2] = f_sig[key][:][SB_range_s][:n_sig_2]
                # f_out[key][n_sig_1+n_bkg_1+n_sig_2:] = f_bkg[key][:][SB_range_b][:n_bkg_2]

                f_out[key][:n_sig_1] = f_sig[key][:][idx_SR_s]
                f_out[key][n_sig_1:n_sig_1+n_bkg_1] = f_bkg[key][:][idx_SR_b]
                f_out[key][n_sig_1+n_bkg_1:n_sig_1+n_bkg_1+n_sig_2] = f_sig[key][:][idx_SB_s]
                f_out[key][n_sig_1+n_bkg_1+n_sig_2:] = f_bkg[key][:][idx_SB_b]
                

        f_out['EVENT/signal'][:n_sig_1+n_bkg_1] = 1
        f_out['EVENT/signal'][n_sig_1+n_bkg_1:] = 0

#### Original dataset

In [7]:
# background.h5 merged from background_03.h5 background_04.h5 files
sig_path = './Sample/HVmodel/data/split_val/signal.h5'
bkg_path = './Sample/HVmodel/data/split_val/background.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

In [8]:
sig_path = './Sample/HVmodel/data/split_val/signal-val.h5'
bkg_path = './Sample/HVmodel/data/split_val/background-val.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_val), int(n_SB_S * r_val), int(n_SR_B * r_val), int(n_SB_B * r_val)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

##### Various Seeds

In [6]:
# print number of events in SR, SB regions
sig_path = './Sample/HVmodel/data/split_val/signal.h5'
bkg_path = './Sample/HVmodel/data/split_val/background.h5'

SRSB_region=[4400, 4700, 5500, 5800]
n_SR_s, n_SB_s, n_SR_b, n_SB_b = 0, 0, 0, 0

with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg: 

    mjj_s = f_sig['EVENT/Mjj'][:]
    mjj_b = f_bkg['EVENT/Mjj'][:]
    SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
    SB_range_s = ((mjj_s > SRSB_region[0]) & (mjj_s < SRSB_region[1])) | ((mjj_s > SRSB_region[2]) & (mjj_s < SRSB_region[3]))
    SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])
    SB_range_b = ((mjj_b > SRSB_region[0]) & (mjj_b < SRSB_region[1])) | ((mjj_b > SRSB_region[2]) & (mjj_b < SRSB_region[3]))

    n_SR_s += SR_range_s.sum()
    n_SB_s += SB_range_s.sum()
    n_SR_b += SR_range_b.sum()
    n_SB_b += SB_range_b.sum()

sig_path = './Sample/HVmodel/data/split_val/signal-val.h5'
bkg_path = './Sample/HVmodel/data/split_val/background-val.h5'
with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg: 

    mjj_s = f_sig['EVENT/Mjj'][:]
    mjj_b = f_bkg['EVENT/Mjj'][:]
    SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
    SB_range_s = ((mjj_s > SRSB_region[0]) & (mjj_s < SRSB_region[1])) | ((mjj_s > SRSB_region[2]) & (mjj_s < SRSB_region[3]))
    SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])
    SB_range_b = ((mjj_b > SRSB_region[0]) & (mjj_b < SRSB_region[1])) | ((mjj_b > SRSB_region[2]) & (mjj_b < SRSB_region[3]))

    n_SR_s += SR_range_s.sum()
    n_SB_s += SB_range_s.sum()
    n_SR_b += SR_range_b.sum()
    n_SB_b += SB_range_b.sum()

print(f'Signal SR: {n_SR_s}')
print(f'Signal SB: {n_SB_s}')
print(f'Background SR: {n_SR_b}')
print(f'Background SB: {n_SB_b}')

Signal SR: 111518
Signal SB: 27100
Background SR: 99145
Background SB: 105000


In [10]:
seeds = [i * 100 + 23 for i in range(1, 11)]
for seed in seeds:

    sig_path = './Sample/HVmodel/data/split_val/signal.h5'
    bkg_path = './Sample/HVmodel/data/split_val/background.h5'

    for i in tqdm(range(11)):
        sensitivity = i
        n_SR_S = sensitivity * np.sqrt(n_SR_B)
        n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

        n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
        output_path = f'./Sample/HVmodel/data/seed_{seed}/mix_sample_{i:.1f}.h5'
        create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, seed=seed)

    sig_path = './Sample/HVmodel/data/split_val/signal-val.h5'
    bkg_path = './Sample/HVmodel/data/split_val/background-val.h5'

    for i in range(11):
        sensitivity = i
        n_SR_S = sensitivity * np.sqrt(n_SR_B)
        n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']
        
        n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_val), int(n_SB_S * r_val), int(n_SR_B * r_val), int(n_SB_B * r_val)
        output_path = f'./Sample/HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_val.h5'
        create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, seed=seed)

100%|██████████| 11/11 [00:48<00:00,  4.38s/it]


#### Luminosity $\times 2$

In [12]:
sig_path = './Sample/HVmodel/data/split_val/signal.h5'
bkg_path = './Sample/HVmodel/data/split_val/background.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

In [13]:
sig_path = './Sample/HVmodel/data/split_val/signal-val.h5'
bkg_path = './Sample/HVmodel/data/split_val/background-val.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S * r_val), int(n_SB_S * r_val), int(n_SR_B * r_val), int(n_SB_B * r_val)
    output_path = f'./Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path, SRSB_region=[4400, 4700, 5500, 5800])

### Make Testing sample

In [3]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test.h5', './Sample/HVmodel/data/split_val/background-test.h5'
SRSB_region=[4400, 4700, 5500, 5800]
n_sig_1, n_bkg_1 = 100000, 100000
nevent = n_sig_1 + n_bkg_1

with h5py.File('./Sample/HVmodel/data/split_val/mix_sample_test-4.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]
        SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
        SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])

        for key in keys:
            f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
            f_out[key][n_sig_1:] = f_bkg[key][:][SR_range_b][:n_bkg_1]

    f_out['EVENT/signal'][:n_sig_1] = 1
    f_out['EVENT/signal'][n_sig_1:] = 0

166927 100173


In [4]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test_2.h5', './Sample/HVmodel/data/split_val/background-test_2.h5'
SRSB_region=[4400, 4700, 5500, 5800]
n_sig_1, n_bkg_1 = 10000, 10000
nevent = n_sig_1 + n_bkg_1

with h5py.File('./Sample/HVmodel/data/split_val/mix_sample_test-2.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]
        SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
        SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])

        for key in keys:
            f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
            f_out[key][n_sig_1:] = f_bkg[key][:][SR_range_b][:n_bkg_1]

    f_out['EVENT/signal'][:n_sig_1] = 1
    f_out['EVENT/signal'][n_sig_1:] = 0

In [4]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test_2.h5', './Sample/HVmodel/data/split_val/background-test_2.h5'
SRSB_region=[4400, 4700, 5500, 5800]
n_sig_1, n_bkg_1 = 20000, 20000
nevent = n_sig_1 + n_bkg_1

with h5py.File('./Sample/HVmodel/data/split_val/mix_sample_test-3.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]
        SR_range_s = (mjj_s > SRSB_region[1]) & (mjj_s < SRSB_region[2])
        SR_range_b = (mjj_b > SRSB_region[1]) & (mjj_b < SRSB_region[2])

        for key in keys:
            f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
            f_out[key][n_sig_1:] = f_bkg[key][:][SR_range_b][:n_bkg_1]

    f_out['EVENT/signal'][:n_sig_1] = 1
    f_out['EVENT/signal'][n_sig_1:] = 0

In [5]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test.h5', './Sample/HVmodel/data/split_val/background-test.h5'
SRSB_region=[4400, 4700, 5500, 5800]
n_sig_SB, n_bkg_SB = 10000, 10000
nevent = n_sig_SB + n_bkg_SB

with h5py.File('./Sample/HVmodel/data/split_val/mix_sample_test-SB.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]

        SB_range_s = ((mjj_s > SRSB_region[0]) & (mjj_s < SRSB_region[1])) | ((mjj_s > SRSB_region[2]) & (mjj_s < SRSB_region[3]))
        SB_range_b = ((mjj_b > SRSB_region[0]) & (mjj_b < SRSB_region[1])) | ((mjj_b > SRSB_region[2]) & (mjj_b < SRSB_region[3]))

        for key in keys:
            f_out[key][:n_sig_SB] = f_sig[key][:][SB_range_s][:n_sig_SB]
            f_out[key][n_sig_SB:] = f_bkg[key][:][SB_range_b][:n_bkg_SB]

    f_out['EVENT/signal'][:n_sig_SB] = 1
    f_out['EVENT/signal'][n_sig_SB:] = 0

In [4]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test.h5', './Sample/HVmodel/data/split_val/background-test.h5'
with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
    mjj_s = f_sig['EVENT/Mjj'][:]
    mjj_b = f_bkg['EVENT/Mjj'][:]
    print(mjj_s.shape, mjj_b.shape)

(71688,) (50236,)


In [ ]:
sig_path, bkg_path  = './Sample/HVmodel/data/split_val/signal-test.h5', './Sample/HVmodel/data/split_val/background-test.h5'

n_sig_SB, n_bkg_SB = 10000, 10000
nevent = n_sig_SB + n_bkg_SB

with h5py.File('./Sample/HVmodel/data/split_val/mix_sample_test-SB.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]

        SB_range_s = ((mjj_s > SRSB_region[0]) & (mjj_s < SRSB_region[1])) | ((mjj_s > SRSB_region[2]) & (mjj_s < SRSB_region[3]))
        SB_range_b = ((mjj_b > SRSB_region[0]) & (mjj_b < SRSB_region[1])) | ((mjj_b > SRSB_region[2]) & (mjj_b < SRSB_region[3]))

        for key in keys:
            f_out[key][:n_sig_SB] = f_sig[key][:][SB_range_s][:n_sig_SB]
            f_out[key][n_sig_SB:] = f_bkg[key][:][SB_range_b][:n_bkg_SB]

    f_out['EVENT/signal'][:n_sig_SB] = 1
    f_out['EVENT/signal'][n_sig_SB:] = 0

## Data augmentation

Sample/physical_augmentation_h5.ipynb

Sample/rotate_jet.ipynb

### Jet rotation

In [1]:
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/rotate_jet/mix_sample_{i:.1f}_jet_aug_3_75x75.h5'
    n = 3
    resolution = 75

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {resolution} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/rotate_jet/mix_sample_0.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/rotate_jet/mix_sample_1.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/rotate_jet/mix_sample_2.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/rotate_jet/mix_sample_3.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/rotate_jet/mix_sample_4.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/rotate_jet/mix_sample_5.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/rotate_jet/mix_sample_6.0_jet_aug_3_75x75.h5 3 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVmodel/data/rota

In [2]:
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/rotate_jet/mix_sample_{i:.1f}_jet_aug_5_75x75.h5'
    n = 5
    resolution = 75

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {resolution} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/rotate_jet/mix_sample_0.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/rotate_jet/mix_sample_1.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/rotate_jet/mix_sample_2.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/rotate_jet/mix_sample_3.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/rotate_jet/mix_sample_4.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/rotate_jet/mix_sample_5.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/rotate_jet/mix_sample_6.0_jet_aug_5_75x75.h5 5 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVmodel/data/rota

In [4]:
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_1_25x25.h5'
    n = 1
    resolution = 25

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {resolution} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/mix_sample_0.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/mix_sample_2.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/mix_sample_3.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/mix_sample_4.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/mix_sample_5.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/mix_sample_6.0_jet_aug_1_25x25.h5 1 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVm

In [5]:
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_3_25x25.h5'
    n = 3
    resolution = 25

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {resolution} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/mix_sample_0.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/mix_sample_2.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/mix_sample_3.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/mix_sample_4.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/mix_sample_5.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/mix_sample_6.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVm

In [6]:
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_5_25x25.h5'
    n = 5
    resolution = 25

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {resolution} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/mix_sample_0.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/mix_sample_2.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/mix_sample_3.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/mix_sample_4.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/mix_sample_5.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/mix_sample_6.0_jet_aug_5_25x25.h5 5 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVm

#### Asymptotic behaviour

In [3]:
i = 1
res = 75
for n in [7, 10, 15, 22, 33]:
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_1.0_jet_aug_7_75x75.h5 7 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_1.0_jet_aug_10_75x75.h5 10 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_1.0_jet_aug_15_75x75.h5 15 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_1.0_jet_aug_22_75x75.h5 22 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_1.0_jet_aug_33_75x75.h5 33 75 &


In [4]:
i = 3
res = 75
for n in [7, 10, 15, 22, 33]:
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_7_75x75.h5 7 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_10_75x75.h5 10 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_15_75x75.h5 15 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_22_75x75.h5 22 75 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_33_75x75.h5 33 75 &


In [5]:
i = 1
res = 25
for n in [7, 10, 15, 22, 33]:
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_1.0_jet_aug_7_25x25.h5 7 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_1.0_jet_aug_10_25x25.h5 10 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_1.0_jet_aug_15_25x25.h5 15 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_1.0_jet_aug_22_25x25.h5 22 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_1.0_jet_aug_33_25x25.h5 33 25 &


In [6]:
i = 3
res = 25
for n in [7, 10, 15, 22, 33]:
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_7_25x25.h5 7 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_10_25x25.h5 10 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_15_25x25.h5 15 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_22_25x25.h5 22 25 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_33_25x25.h5 33 25 &


#### Rotation range

In [1]:
n = 5
res = 25
angle = 90
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} {angle} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/25x25/range_90/mix_sample_0.0_jet_aug_5_25x25.h5 5 25 90 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/range_90/mix_sample_1.0_jet_aug_5_25x25.h5 5 25 90 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/25x25/range_90/mix_sample_2.0_jet_aug_5_25x25.h5 5 25 90 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/range_90/mix_sample_3.0_jet_aug_5_25x25.h5 5 25 90 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/25x25/range_90/mix_sample_4.0_jet_aug_5_25x25.h5 5 25 90 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/25x25/range_90/mix_sample_5.0_jet_aug_5_25x25.h5 5 25 90 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/25

In [3]:
n = 5
res = 25
angle = 5
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} {angle} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/mix_sample_0.0_jet_aug_5_25x25.h5 5 25 5 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/mix_sample_1.0_jet_aug_5_25x25.h5 5 25 5 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/mix_sample_2.0_jet_aug_5_25x25.h5 5 25 5 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/mix_sample_3.0_jet_aug_5_25x25.h5 5 25 5 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/mix_sample_4.0_jet_aug_5_25x25.h5 5 25 5 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/mix_sample_5.0_jet_aug_5_25x25.h5 5 25 5 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/25x25/range_5/

In [2]:
n = 5
res = 25
angle = 45
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} {angle} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/25x25/range_45/mix_sample_0.0_jet_aug_5_25x25.h5 5 25 45 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/range_45/mix_sample_1.0_jet_aug_5_25x25.h5 5 25 45 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/25x25/range_45/mix_sample_2.0_jet_aug_5_25x25.h5 5 25 45 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/range_45/mix_sample_3.0_jet_aug_5_25x25.h5 5 25 45 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/25x25/range_45/mix_sample_4.0_jet_aug_5_25x25.h5 5 25 45 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/25x25/range_45/mix_sample_5.0_jet_aug_5_25x25.h5 5 25 45 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/25

In [1]:
n = 5
res = 25
angle = 1
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} {angle} &'
    print(cmd)

python rotate_jet.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/mix_sample_0.0_jet_aug_5_25x25.h5 5 25 1 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/mix_sample_1.0_jet_aug_5_25x25.h5 5 25 1 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/mix_sample_2.0_jet_aug_5_25x25.h5 5 25 1 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/mix_sample_3.0_jet_aug_5_25x25.h5 5 25 1 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/mix_sample_4.0_jet_aug_5_25x25.h5 5 25 1 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/mix_sample_5.0_jet_aug_5_25x25.h5 5 25 1 &
python rotate_jet.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/jet_rotation/25x25/range_1/

#### Fix rotation angles

In [3]:
res = 25
angles = '-5_5'
for i in range(11):

    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/jet_rotation/{res}x{res}/angles_{angles}/mix_sample_{i:.1f}_jet_aug_angles_{angles}_{res}x{res}.h5'

    cmd = f'python rotate_jet.py {h5_file} {output_path} &'
    cmd = f'python rotate_jet_w_angles.py {h5_file} {output_path} {angles} {res} &'
    print(cmd)

python rotate_jet_w_angles.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_0.0_jet_aug_angle_-5_5_25x25.h5 -5_5 25 &
python rotate_jet_w_angles.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_1.0_jet_aug_angle_-5_5_25x25.h5 -5_5 25 &
python rotate_jet_w_angles.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_2.0_jet_aug_angle_-5_5_25x25.h5 -5_5 25 &
python rotate_jet_w_angles.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_3.0_jet_aug_angle_-5_5_25x25.h5 -5_5 25 &
python rotate_jet_w_angles.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_4.0_jet_aug_angle_-5_5_25x25.h5 -5_5 25 &
python rotate_jet_w_angles.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_5.0_jet_au

### $p_\text{T}$ smearing + jet rotation

In [7]:
for i in range(11):
    n = 1
    res = 25
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {resolution} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_0.0_aug_1_25x25.h5 1 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_1_25x25.h5 1 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_2.0_aug_1_25x25.h5 1 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_1_25x25.h5 1 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_4.0_aug_1_25x25.h5 1 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_5.0_aug_1_25x25.h5 1 25 &
python pT_smearing_jet_rotation.py

In [8]:
for i in range(11):
    n = 3
    res = 25
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_0.0_aug_3_25x25.h5 3 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_3_25x25.h5 3 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_2.0_aug_3_25x25.h5 3 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_3_25x25.h5 3 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_4.0_aug_3_25x25.h5 3 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_5.0_aug_3_25x25.h5 3 25 &
python pT_smearing_jet_rotation.py

In [2]:
for i in range(11):
    n = 3
    res = 75
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_0.0_aug_3_75x75.h5 3 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_3_75x75.h5 3 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_2.0_aug_3_75x75.h5 3 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_3_75x75.h5 3 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_4.0_aug_3_75x75.h5 3 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_5.0_aug_3_75x75.h5 3 75 &
python pT_smearing_jet_rotation.py

#### Asymptotic behaviour

In [1]:
i = 3
for n in [5, 7, 10, 15, 22, 33]:
    res = 75
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_5_75x75.h5 5 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_7_75x75.h5 7 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_10_75x75.h5 10 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_15_75x75.h5 15 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_22_75x75.h5 22 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_33_75x75.h5 33 75 &


In [4]:
i = 1
for n in [5, 7, 10, 15, 22, 33]:
    res = 75
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_5_75x75.h5 5 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_7_75x75.h5 7 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_10_75x75.h5 10 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_15_75x75.h5 15 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_22_75x75.h5 22 75 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_33_75x75.h5 33 75 &


In [5]:
i = 3
for n in [5, 7, 10, 15, 22, 33]:
    res = 25
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_5_25x25.h5 5 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_7_25x25.h5 7 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_10_25x25.h5 10 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_15_25x25.h5 15 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_22_25x25.h5 22 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_33_25x25.h5 33 25 &


In [6]:
i = 1
for n in [5, 7, 10, 15, 22, 33]:
    res = 25
    h5_file = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.h5'

    cmd = f'python pT_smearing_jet_rotation.py {h5_file} {output_path} {n} {res} &'
    print(cmd)

python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_5_25x25.h5 5 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_7_25x25.h5 7 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_10_25x25.h5 10 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_15_25x25.h5 15 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_22_25x25.h5 22 25 &
python pT_smearing_jet_rotation.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_33_25x25.h5 33 25 &


### Different training dataset

In [4]:
seeds = [i * 100 + 23 for i in range(1, 10)]
for seed in seeds:

    for i in range(11):

        res = 25
        n = 3
        h5_file = f'./HVmodel/data/seed_{seed}/mix_sample_{i:.1f}.h5'
        output_path = f'./HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.h5'

        cmd = f'python rotate_jet.py {h5_file} {output_path} {n} {res} &'
        print(cmd)

python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_0.0.h5 ./HVmodel/data/seed_123/mix_sample_0.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_1.0.h5 ./HVmodel/data/seed_123/mix_sample_1.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_2.0.h5 ./HVmodel/data/seed_123/mix_sample_2.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_3.0.h5 ./HVmodel/data/seed_123/mix_sample_3.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_4.0.h5 ./HVmodel/data/seed_123/mix_sample_4.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_5.0.h5 ./HVmodel/data/seed_123/mix_sample_5.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_6.0.h5 ./HVmodel/data/seed_123/mix_sample_6.0_jet_aug_3_25x25.h5 3 25 &
python rotate_jet.py ./HVmodel/data/seed_123/mix_sample_7.0.h5 ./HVmodel/data/seed_123/mix_sample_7.0_je

## From h5 to npy

### Resolution $75 \times 75$

#### Original mix sample

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/split_val/mix_sample_0.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/split_val/mix_sample_1.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/split_val/mix_sample_2.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/split_val/mix_sample_3.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/split_val/mix_sample_4.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/split_val/mix_sample_5.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/split_val/mix_sample_6.0_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVmodel/data/split_val/mix_sample_7.0_75x75.npy 75 &
python from_h5_t

#### Validation dataset

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_val.h5 ./HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_val.h5 ./HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_val.h5 ./HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_val.h5 ./HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_val.h5 ./HVmodel/data/split_val/mix_sample_4.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_val.h5 ./HVmodel/data/split_val/mix_sample_5.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_val.h5 ./HVmodel/data/split_val/mix_sample_6.0_val_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_val.h5 ./HVmodel/d

#### Testing dataset

In [5]:
h5_path = f'./HVmodel/data/split_val/mix_sample_test.h5'
output_path = f'./HVmodel/data/split_val/mix_sample_test_75x75.npy'
resolution = 75

cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_test.h5 ./HVmodel/data/split_val/mix_sample_test_75x75.npy 75 &


#### Copy: +1

In [6]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_copy_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/s

#### $\eta-\phi$ augmentation

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_s

In [1]:
for i in range(11):
    h5_path = f'./HVmodel/data/eta_phi_smearing/mix_sample_{i:.1f}_aug_3.h5'
    output_path = f'./HVmodel/data/eta_phi_smearing/75x75/mix_sample_{i:.1f}_aug_3_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_0.0_aug_3.h5 ./HVmodel/data/eta_phi_smearing/75x75/mix_sample_0.0_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_1.0_aug_3.h5 ./HVmodel/data/eta_phi_smearing/75x75/mix_sample_1.0_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_2.0_aug_3.h5 ./HVmodel/data/eta_phi_smearing/75x75/mix_sample_2.0_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_3.0_aug_3.h5 ./HVmodel/data/eta_phi_smearing/75x75/mix_sample_3.0_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_4.0_aug_3.h5 ./HVmodel/data/eta_phi_smearing/75x75/mix_sample_4.0_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_5.0_aug_3.h5 ./HVmodel/data/eta_phi_smearing/75x75/mix_sample_5.0_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_smearing/mix_sample_6.

#### Luminosity: $\times 2$

In [5]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_x2.h5 ./HVmodel/data/split_val/mix_sample_0.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_x2.h5 ./HVmodel/data/split_val/mix_sample_1.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_x2.h5 ./HVmodel/data/split_val/mix_sample_2.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_x2.h5 ./HVmodel/data/split_val/mix_sample_3.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_x2.h5 ./HVmodel/data/split_val/mix_sample_4.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_x2.h5 ./HVmodel/data/split_val/mix_sample_5.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_x2.h5 ./HVmodel/data/split_val/mix_sample_6.0_x2_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_x2.h5 ./HVmodel/data/split_val/m

#### Smearing scale

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_6.0_aug_1.h5 ./HVmodel/da

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_6.0_aug_1.h5 ./HVmodel/da

#### $p_{\text{T}}$ augmentation

In [7]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_1_75x75.npy 75 &
python from_h

In [1]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_3.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_3_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_3_75x75.npy 75 &
python from_h

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_5.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_5_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_5_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_5_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_5_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_5_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_5_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_5_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_5.h5 ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_5_75x75.npy 75 &
python from_h

#### Asymptotic behaviour

In [4]:
i = 1
for n in [7, 10, 15, 22]:

    res = 75
    h5_path = f'./HVmodel/data/pt_smearing/mix_sample_{i:.1f}_pt_aug_{n}.h5'
    output_path = f'./HVmodel/data/pt_smearing/{res}x{res}/mix_sample_{i:.1f}_pt_aug_{n}_{res}x{res}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {res} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_7.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_7_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_10.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_10_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_15.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_15_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_22.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_22_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_33.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_33_75x75.npy 75 &


In [3]:
i = 3
for n in [7, 10, 15, 22]:

    res = 75
    h5_path = f'./HVmodel/data/pt_smearing/mix_sample_{i:.1f}_pt_aug_{n}.h5'
    output_path = f'./HVmodel/data/pt_smearing/{res}x{res}/mix_sample_{i:.1f}_pt_aug_{n}_{res}x{res}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {res} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_7.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_7_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_10.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_10_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_15.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_15_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_22.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_22_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_33.h5 ./HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_33_75x75.npy 75 &


#### $\eta-\phi$ augmentation + $p_{\text{T}}$ augmentation

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_eta_phi_pt_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_eta_phi_pt_aug_1_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_eta_phi_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_eta_phi_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_eta_phi_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_eta_phi_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_eta_phi_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_eta_phi_pt_aug_1_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_samp

In [1]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_eta_phi_pt_aug_3.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_eta_phi_pt_aug_3_75x75.npy'
    resolution = 75

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_0.0_eta_phi_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_1.0_eta_phi_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_2.0_eta_phi_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_3.0_eta_phi_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_4.0_eta_phi_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/split_val/mix_sample_5.0_eta_phi_pt_aug_3_75x75.npy 75 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_samp

### Resolution $25 \times 25$

#### Original mix sample

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0.h5 ./HVmodel/data/split_val/mix_sample_0.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0.h5 ./HVmodel/data/split_val/mix_sample_1.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0.h5 ./HVmodel/data/split_val/mix_sample_2.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0.h5 ./HVmodel/data/split_val/mix_sample_3.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0.h5 ./HVmodel/data/split_val/mix_sample_4.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0.h5 ./HVmodel/data/split_val/mix_sample_5.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0.h5 ./HVmodel/data/split_val/mix_sample_6.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0.h5 ./HVmodel/data/split_val/mix_sample_7.0_25x25.npy 25 &
python from_h5_t

#### Validation dataset

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_val.h5 ./HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_val.h5 ./HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_val.h5 ./HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_val.h5 ./HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_val.h5 ./HVmodel/data/split_val/mix_sample_4.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_val.h5 ./HVmodel/data/split_val/mix_sample_5.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_val.h5 ./HVmodel/data/split_val/mix_sample_6.0_val_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_val.h5 ./HVmodel/d

#### Testing dataset

In [4]:
h5_path = f'./HVmodel/data/split_val/mix_sample_test.h5'
output_path = f'./HVmodel/data/split_val/mix_sample_test_25x25.npy'
resolution = 25

cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_test.h5 ./HVmodel/data/split_val/mix_sample_test_25x25.npy 25 &


In [6]:
res = 25
h5_path = f'./HVmodel/data/split_val/mix_sample_test-SB.h5'
output_path = f'./HVmodel/data/split_val/{res}x{res}/mix_sample_test-SB_{res}x{res}.npy'

cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {res} &'
print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_test-SB.h5 ./HVmodel/data/split_val/25x25/mix_sample_test-SB_25x25.npy 25 &


#### Duplicated

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_copy_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_copy_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/s

In [2]:
res = 25
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_5.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_copy_5_{res}x{res}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {res} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_0.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_1.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_2.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_3.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_4.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_5.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_copy_5.h5 ./HVmodel/data/split_val/mix_sample_6.0_copy_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/s

#### Luminosity: $\times 2$

In [17]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)
    
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_x2.h5 ./HVmodel/data/split_val/mix_sample_0.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_x2.h5 ./HVmodel/data/split_val/mix_sample_1.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_x2.h5 ./HVmodel/data/split_val/mix_sample_2.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_x2.h5 ./HVmodel/data/split_val/mix_sample_3.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_x2.h5 ./HVmodel/data/split_val/mix_sample_4.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_x2.h5 ./HVmodel/data/split_val/mix_sample_5.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_x2.h5 ./HVmodel/data/split_val/mix_sample_6.0_x2_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_7.0_x2.h5 ./HVmodel/data/split_val/m

#### $\eta-\phi$ augmentation

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_aug_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_s

In [4]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_0.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_1.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_2.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_3.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_4.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_5.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_aug_3.h5 ./HVmodel/data/split_val/mix_sample_6.0_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_s

#### Smearing scale

In [1]:
for i in range(11):
    h5_path = f'./HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_3.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_4.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1.h5 ./HVmodel/data/smearing_scale_02/mix_sample_5.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_02/mix_sample_6.0_aug_1.h5 ./HVmodel/da

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_3.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_4.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1.h5 ./HVmodel/data/smearing_scale_05/mix_sample_5.0_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/smearing_scale_05/mix_sample_6.0_aug_1.h5 ./HVmodel/da

#### $p_{\text{T}}$ augmentation

In [8]:
for i in range(11):
    h5_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_1.h5'
    output_path = f'./HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_0.0_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_1.0_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_2.0_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_3.0_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_4.0_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_5.0_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_1.h5 ./HVmodel/data/split_val/mix_sample_6.0_pt_aug_1_25x25.npy 25 &
python from_h

In [2]:
for i in range(11):
    h5_path = f'./HVmodel/data/pt_smearing/mix_sample_{i:.1f}_pt_aug_3.h5'
    output_path = f'./HVmodel/data/pt_smearing/25x25/mix_sample_{i:.1f}_pt_aug_3_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_0.0_pt_aug_3.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_0.0_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_3.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_2.0_pt_aug_3.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_2.0_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_3.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_4.0_pt_aug_3.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_4.0_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_5.0_pt_aug_3.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_5.0_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_6.0_pt_aug_3.h5 ./HVmodel/data/

In [3]:
for i in range(11):
    h5_path = f'./HVmodel/data/pt_smearing/mix_sample_{i:.1f}_pt_aug_5.h5'
    output_path = f'./HVmodel/data/pt_smearing/25x25/mix_sample_{i:.1f}_pt_aug_5_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_0.0_pt_aug_5.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_0.0_pt_aug_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_5.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_2.0_pt_aug_5.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_2.0_pt_aug_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_5.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_4.0_pt_aug_5.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_4.0_pt_aug_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_5.0_pt_aug_5.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_5.0_pt_aug_5_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_6.0_pt_aug_5.h5 ./HVmodel/data/

#### Asymptotic behaviour

In [13]:
i = 1
for n in [7, 10, 15, 22]:
    res = 25
    h5_path = f'./HVmodel/data/pt_smearing/mix_sample_{i:.1f}_pt_aug_{n}.h5'
    output_path = f'./HVmodel/data/pt_smearing/{res}x{res}/mix_sample_{i:.1f}_pt_aug_{n}_{res}x{res}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {res} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_7.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_7_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_10.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_10_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_15.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_15_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_1.0_pt_aug_22.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_22_25x25.npy 25 &


In [7]:
i = 3
for n in [7, 10, 15, 22, 33]:

    res = 25
    h5_path = f'./HVmodel/data/pt_smearing/mix_sample_{i:.1f}_pt_aug_{n}.h5'
    output_path = f'./HVmodel/data/pt_smearing/{res}x{res}/mix_sample_{i:.1f}_pt_aug_{n}_{res}x{res}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {res} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_7.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_7_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_10.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_10_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_15.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_15_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_22.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_22_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/pt_smearing/mix_sample_3.0_pt_aug_33.h5 ./HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_33_25x25.npy 25 &


#### $\eta-\phi$ augmentation + $p_{\text{T}}$ augmentation

In [5]:
for i in range(11):
    h5_path = f'./HVmodel/data/eta_phi_pt_smearing/mix_sample_{i:.1f}_eta_phi_pt_aug_1.h5'
    output_path = f'./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_{i:.1f}_eta_phi_pt_aug_1_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_0.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_0.0_eta_phi_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_1.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_1.0_eta_phi_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_2.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_2.0_eta_phi_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_3.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_3.0_eta_phi_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_4.0_eta_phi_pt_aug_1.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_4.0_eta_phi_pt_aug_1_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_5.0_eta_phi_pt

In [6]:
for i in range(11):
    h5_path = f'./HVmodel/data/eta_phi_pt_smearing/mix_sample_{i:.1f}_eta_phi_pt_aug_3.h5'
    output_path = f'./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_{i:.1f}_eta_phi_pt_aug_3_25x25.npy'
    resolution = 25

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_0.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_0.0_eta_phi_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_1.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_1.0_eta_phi_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_2.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_2.0_eta_phi_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_3.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_3.0_eta_phi_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_4.0_eta_phi_pt_aug_3.h5 ./HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_4.0_eta_phi_pt_aug_3_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/eta_phi_pt_smearing/mix_sample_5.0_eta_phi_pt

#### Different training datasets

In [11]:
seeds = [i * 100 + 23 for i in range(1, 10)]
for seed in seeds:

    for i in range(11):
        h5_path = f'./HVmodel/data/seed_{seed}/mix_sample_{i:.1f}.h5'
        output_path = f'./HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_25x25.npy'
        resolution = 25

        cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
        print(cmd)

    for i in range(11):
        h5_path = f'./HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_val.h5'
        output_path = f'./HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_val_25x25.npy'
        resolution = 25

        cmd = f'python from_h5_to_npy.py {h5_path} {output_path} {resolution} &'
        print(cmd)

python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_0.0.h5 ./HVmodel/data/seed_123/mix_sample_0.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_1.0.h5 ./HVmodel/data/seed_123/mix_sample_1.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_2.0.h5 ./HVmodel/data/seed_123/mix_sample_2.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_3.0.h5 ./HVmodel/data/seed_123/mix_sample_3.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_4.0.h5 ./HVmodel/data/seed_123/mix_sample_4.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_5.0.h5 ./HVmodel/data/seed_123/mix_sample_5.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_6.0.h5 ./HVmodel/data/seed_123/mix_sample_6.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmodel/data/seed_123/mix_sample_7.0.h5 ./HVmodel/data/seed_123/mix_sample_7.0_25x25.npy 25 &
python from_h5_to_npy.py ./HVmod

# CNN Training

## Original mix dataset

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_75x75 "Sensitivity: 0.0, Resolution: 75x75"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_75x75 "Sensitivity: 1.0, Resolution: 75x75"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_75x75 "Sensitivity: 2.0, Resolution: 75x75"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_3.0_75x75 "Sensitivity: 3.0, Resolution: 75x75"


In [3]:
res = 25
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_25x25 "Sensitivity: 0.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_25x25 "Sensitivity: 1.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_25x25 "Sensitivity: 2.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_s

## Duplicated

In [14]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_copy_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Copy: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_copy_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_copy_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_copy_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_copy_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_va

In [15]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_copy_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_copy_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Copy: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_copy_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_copy_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_copy_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, Copy: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_copy_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_va

In [3]:
res = 25
n = 5
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_copy_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_copy_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Copy: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_copy_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_copy_5_25x25 "Sensitivity: 0.0, Resolution: 25x25, Copy: +5"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_copy_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_copy_5_25x25 "Sensitivity: 1.0, Resolution: 25x25, Copy: +5"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_copy_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_copy_5_25x25 "Sensitivity: 2.0, Resolution: 25x25, Copy: +5"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_copy_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val

## $\eta-\phi$ augmentation

In [5]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_aug_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Augmentation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_aug_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_aug_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_aug_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.npy ../Sample/HVmo

In [6]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_aug_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_aug_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_aug_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_aug_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmo

In [4]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/eta_phi_smearing/75x75/mix_sample_{i:.1f}_aug_3_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_aug_3_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Augmentation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/eta_phi_smearing/75x75/mix_sample_0.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_aug_3_75x75 "Sensitivity: 0.0, Resolution: 75x75, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_smearing/75x75/mix_sample_1.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_aug_3_75x75 "Sensitivity: 1.0, Resolution: 75x75, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_smearing/75x75/mix_sample_2.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_aug_3_75x75 "Sensitivity: 2.0, Resolution: 75x75, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_smearing/75x75/mix_sample_3.0_aug_3_75x75.npy ../Sample/HVmodel/data/s

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_aug_3_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_aug_3_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_aug_3_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_aug_3_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_aug_3_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.npy ../Sample/HVmo

## Luminosity: $\times 2$

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_x2_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_x2_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_x2_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_x2_75x75 "Sensitivity: 0.0, Resolution: 75x75, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_x2_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_x2_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_x2_75x75 "Sensitivity: 1.0, Resolution: 75x75, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_x2_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_x2_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_x2_75x75 "Sensitivity: 2.0, Resolution: 75x75, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_x2_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_x2_val_75x75.npy ../Sample/HVmodel/data/split_

In [18]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_x2_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_x2_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_x2_25x25 "Sensitivity: 0.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_x2_25x25 "Sensitivity: 1.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_x2_25x25 "Sensitivity: 2.0, Resolution: 25x25, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_x2_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_x2_val_25x25.npy ../Sample/HVmodel/data/split_

## Smearing scale

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_aug_1_std_02_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Augmentation: +1, Smearing: 0.2'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_aug_1_std_02_75x75 "Sensitivity: 0.0, Resolution: 75x75, Augmentation: +1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_aug_1_std_02_75x75 "Sensitivity: 1.0, Resolution: 75x75, Augmentation: +1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_aug_1_std_02_75x75 "Sensitivity: 2.0, Resolution: 75x75, Augmentation: +1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sampl

In [4]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_aug_1_std_05_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Augmentation: +1, Smearing: 0.5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_aug_1_std_05_75x75 "Sensitivity: 0.0, Resolution: 75x75, Augmentation: +1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_aug_1_std_05_75x75 "Sensitivity: 1.0, Resolution: 75x75, Augmentation: +1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_aug_1_std_05_75x75 "Sensitivity: 2.0, Resolution: 75x75, Augmentation: +1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sampl

In [5]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/smearing_scale_02/mix_sample_{i:.1f}_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_aug_1_std_02_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: +1, Smearing: 0.2'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_0.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_aug_1_std_02_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: +1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_1.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_aug_1_std_02_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: +1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sample_2.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_aug_1_std_02_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: +1, Smearing: 0.2"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_02/mix_sampl

In [6]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/smearing_scale_05/mix_sample_{i:.1f}_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_aug_1_std_05_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Augmentation: +1, Smearing: 0.5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_0.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_aug_1_std_05_25x25 "Sensitivity: 0.0, Resolution: 25x25, Augmentation: +1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_1.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_aug_1_std_05_25x25 "Sensitivity: 1.0, Resolution: 25x25, Augmentation: +1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sample_2.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_aug_1_std_05_25x25 "Sensitivity: 2.0, Resolution: 25x25, Augmentation: +1, Smearing: 0.5"
python train_CNN.py ../Sample/HVmodel/data/smearing_scale_05/mix_sampl

## $p_{\text{T}}$ augmentation

In [9]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_pt_aug_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, pT smearing: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_pt_aug_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pt_aug_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_pt_aug_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_3_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_pt_aug_3_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, pT smearing: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_pt_aug_3_75x75 "Sensitivity: 0.0, Resolution: 75x75, pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pt_aug_3_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_pt_aug_3_75x75 "Sensitivity: 2.0, Resolution: 75x75, pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.

In [4]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_5_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_pt_aug_5_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, pT smearing: +5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_pt_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_pt_aug_5_75x75 "Sensitivity: 0.0, Resolution: 75x75, pT smearing: +5"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_pt_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pt_aug_5_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing: +5"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_pt_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_pt_aug_5_75x75 "Sensitivity: 2.0, Resolution: 75x75, pT smearing: +5"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_pt_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_75x75.

In [10]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_pt_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_pt_aug_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, pT smearing: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_pt_aug_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pt_aug_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_pt_aug_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_3.0_val_25x25.

In [5]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_{i:.1f}_pt_aug_3_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_pt_aug_3_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, pT smearing: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_0.0_pt_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_pt_aug_3_25x25 "Sensitivity: 0.0, Resolution: 25x25, pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pt_aug_3_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_2.0_pt_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_pt_aug_3_25x25 "Sensitivity: 2.0, Resolution: 25x25, pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_3_25x25.npy ../Sample/HVmodel/data/spl

In [6]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_{i:.1f}_pt_aug_5_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_pt_aug_5_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, pT smearing: +5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_0.0_pt_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_pt_aug_5_25x25 "Sensitivity: 0.0, Resolution: 25x25, pT smearing: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pt_aug_5_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_2.0_pt_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_pt_aug_5_25x25 "Sensitivity: 2.0, Resolution: 25x25, pT smearing: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_5_25x25.npy ../Sample/HVmodel/data/spl

## $\eta-\phi$ augmentation + $p_{\text{T}}$ augmentation

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_eta_phi_pt_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_eta_phi_pt_aug_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, eta phi pT smearing: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_eta_phi_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_eta_phi_pt_aug_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, eta phi pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_eta_phi_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_eta_phi_pt_aug_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, eta phi pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_eta_phi_pt_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_eta_phi_pt_aug_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, eta phi pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_eta_p

In [4]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_eta_phi_pt_aug_3_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_eta_phi_pt_aug_3_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, eta phi pT smearing: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_0.0_eta_phi_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_eta_phi_pt_aug_3_75x75 "Sensitivity: 0.0, Resolution: 75x75, eta phi pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_1.0_eta_phi_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_eta_phi_pt_aug_3_75x75 "Sensitivity: 1.0, Resolution: 75x75, eta phi pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_2.0_eta_phi_pt_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_eta_phi_pt_aug_3_75x75 "Sensitivity: 2.0, Resolution: 75x75, eta phi pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/split_val/mix_sample_3.0_eta_p

In [7]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_{i:.1f}_eta_phi_pt_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_eta_phi_pt_aug_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, eta phi pT smearing: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_0.0_eta_phi_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_eta_phi_pt_aug_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, eta phi pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_1.0_eta_phi_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_eta_phi_pt_aug_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, eta phi pT smearing: +1"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_2.0_eta_phi_pt_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_eta_phi_pt_aug_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, eta phi pT smearing: +1"
python train_CNN.py ../Sa

In [8]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_{i:.1f}_eta_phi_pt_aug_3_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_eta_phi_pt_aug_3_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, eta phi pT smearing: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_0.0_eta_phi_pt_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_eta_phi_pt_aug_3_25x25 "Sensitivity: 0.0, Resolution: 25x25, eta phi pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_1.0_eta_phi_pt_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_eta_phi_pt_aug_3_25x25 "Sensitivity: 1.0, Resolution: 25x25, eta phi pT smearing: +3"
python train_CNN.py ../Sample/HVmodel/data/eta_phi_pt_smearing/25x25/mix_sample_2.0_eta_phi_pt_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_eta_phi_pt_aug_3_25x25 "Sensitivity: 2.0, Resolution: 25x25, eta phi pT smearing: +3"
python train_CNN.py ../Sa

## Jet rotation

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/rotate_jet/mix_sample_{i:.1f}_jet_aug_1_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_jet_aug_1_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Jet rotation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_0.0_jet_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_jet_aug_1_75x75 "Sensitivity: 0.0, Resolution: 75x75, Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_1.0_jet_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_jet_aug_1_75x75 "Sensitivity: 1.0, Resolution: 75x75, Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_2.0_jet_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_jet_aug_1_75x75 "Sensitivity: 2.0, Resolution: 75x75, Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_3.0_jet_aug_1_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_

In [1]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/rotate_jet/mix_sample_{i:.1f}_jet_aug_3_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_jet_aug_3_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Jet rotation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_0.0_jet_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_jet_aug_3_75x75 "Sensitivity: 0.0, Resolution: 75x75, Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_1.0_jet_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_jet_aug_3_75x75 "Sensitivity: 1.0, Resolution: 75x75, Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_2.0_jet_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_jet_aug_3_75x75 "Sensitivity: 2.0, Resolution: 75x75, Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_3.0_jet_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_

In [2]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/rotate_jet/mix_sample_{i:.1f}_jet_aug_5_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Jet rotation: +5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_0.0_jet_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_jet_aug_5_75x75 "Sensitivity: 0.0, Resolution: 75x75, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_1.0_jet_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_jet_aug_5_75x75 "Sensitivity: 1.0, Resolution: 75x75, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_2.0_jet_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_jet_aug_5_75x75 "Sensitivity: 2.0, Resolution: 75x75, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/rotate_jet/mix_sample_3.0_jet_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_

In [7]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_jet_aug_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Jet rotation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_0.0_jet_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_jet_aug_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_2.0_jet_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_jet_aug_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_3.0_jet_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix

In [8]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_3_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_jet_aug_3_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Jet rotation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_0.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_jet_aug_3_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_3_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_2.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_jet_aug_3_25x25 "Sensitivity: 2.0, Resolution: 25x25, Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_3.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix

In [9]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_5_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Jet rotation: +5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_0.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_jet_aug_5_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_5_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_2.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_jet_aug_5_25x25 "Sensitivity: 2.0, Resolution: 25x25, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_3.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/mix

## Asymptotic behaviour

### Jet rotation

In [10]:
i = 3
res = 75
for n in [7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_7_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_jet_aug_7_75x75 "Sensitivity: 3.0, Resolution: 75x75, Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_10_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_jet_aug_10_75x75 "Sensitivity: 3.0, Resolution: 75x75, Jet rotation: +10"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/75x75/mix_sample_3.0_jet_aug_15_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_jet_aug_15_75x75 "Sensitivity: 3.0, Resolution: 75x75, Jet rotation: +15"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/75x75/mix_s

In [9]:
i = 3
res = 25
for n in [7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_7_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_jet_aug_7_25x25 "Sensitivity: 3.0, Resolution: 25x25, Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_10_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_jet_aug_10_25x25 "Sensitivity: 3.0, Resolution: 25x25, Jet rotation: +10"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_sample_3.0_jet_aug_15_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_jet_aug_15_25x25 "Sensitivity: 3.0, Resolution: 25x25, Jet rotation: +15"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_s

In [4]:
i = 1
for n in [7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_{n}_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_jet_aug_{n}_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_7_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_jet_aug_7_75x75 "Sensitivity: 1.0, Resolution: 75x75, Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_10_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_jet_aug_10_75x75 "Sensitivity: 1.0, Resolution: 75x75, Jet rotation: +10"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_15_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_jet_aug_15_75x75 "Sensitivity: 1.0, Resolution: 75x75, Jet rotation: +15"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_22_75x75.npy ../Sample/HVmodel/data/split_

In [5]:
i = 1
for n in [7, 10, 15, 22]:
    train_file = f'../Sample/HVmodel/data/jet_rotation/mix_sample_{i:.1f}_jet_aug_{n}_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_jet_aug_{n}_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_7_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_7_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_10_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_10_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +10"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_15_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_15_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +15"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/mix_sample_1.0_jet_aug_22_25x25.npy ../Sample/HVmodel/data/split_

### $p_\text{T}$ smearing

In [1]:
i = 1
for n in [7, 10, 15, 22]:
    train_file = f'../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_{i:.1f}_pt_aug_{n}_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_pt_aug_{n}_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, pT smearing: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_7_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pt_aug_7_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_10_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pt_aug_10_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing: +10"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_15_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pt_aug_15_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing: +15"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_1.0_pt_aug_22_75x75.npy ../Sample/HVmodel/d

In [11]:
i = 3
res = 75
for n in [7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/pt_smearing/{res}x{res}/mix_sample_{i:.1f}_pt_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_pt_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, pT smearing: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_7_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_pt_aug_7_75x75 "Sensitivity: 3.0, Resolution: 75x75, pT smearing: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_10_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_pt_aug_10_75x75 "Sensitivity: 3.0, Resolution: 75x75, pT smearing: +10"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_aug_15_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_pt_aug_15_75x75 "Sensitivity: 3.0, Resolution: 75x75, pT smearing: +15"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/75x75/mix_sample_3.0_pt_

In [3]:
i = 1
for n in [7, 10, 15, 22]:
    train_file = f'../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_{i:.1f}_pt_aug_{n}_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_pt_aug_{n}_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, pT smearing: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_7_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pt_aug_7_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_10_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pt_aug_10_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing: +10"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_15_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pt_aug_15_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing: +15"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_1.0_pt_aug_22_25x25.npy ../Sample/HVmodel/d

In [8]:
i = 3
res = 25
for n in [7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/pt_smearing/{res}x{res}/mix_sample_{i:.1f}_pt_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_pt_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, pT smearing: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_7_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_pt_aug_7_25x25 "Sensitivity: 3.0, Resolution: 25x25, pT smearing: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_10_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_pt_aug_10_25x25 "Sensitivity: 3.0, Resolution: 25x25, pT smearing: +10"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_aug_15_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_pt_aug_15_25x25 "Sensitivity: 3.0, Resolution: 25x25, pT smearing: +15"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing/25x25/mix_sample_3.0_pt_

### $p_\text{T}$ smearing + jet rotation

In [10]:
i = 1
res = 75
for n in [5, 7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_pt_jet_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_1.0_pt_jet_aug_5_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing + Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_7_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_1.0_pt_jet_aug_7_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing + Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_10_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_1.0_pt_jet_aug_10_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing + Jet rotation: 

In [7]:
i = 3
res = 75
for n in [5, 7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_pt_jet_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_5_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_pt_jet_aug_5_75x75 "Sensitivity: 3.0, Resolution: 75x75, pT smearing + Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_7_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_pt_jet_aug_7_75x75 "Sensitivity: 3.0, Resolution: 75x75, pT smearing + Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_3.0_aug_10_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_3.0_val_75x75.npy ../Sample/HVmodel/data/split_val/75x75/mix_sample_test_75x75.npy SB_3.0_pt_jet_aug_10_75x75 "Sensitivity: 3.0, Resolution: 75x75, pT smearing + Jet rotation: 

In [8]:
i = 1
res = 25
for n in [5, 7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_pt_jet_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_pt_jet_aug_5_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing + Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_7_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_pt_jet_aug_7_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing + Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_10_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_pt_jet_aug_10_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing + Jet rotation: 

In [9]:
i = 3
res = 25
for n in [5, 7, 10, 15, 22, 33]:
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/mix_sample_{i:.1f}_aug_{n}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_pt_jet_aug_{n}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{n}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_pt_jet_aug_5_25x25 "Sensitivity: 3.0, Resolution: 25x25, pT smearing + Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_7_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_pt_jet_aug_7_25x25 "Sensitivity: 3.0, Resolution: 25x25, pT smearing + Jet rotation: +7"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_3.0_aug_10_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_3.0_pt_jet_aug_10_25x25 "Sensitivity: 3.0, Resolution: 25x25, pT smearing + Jet rotation: 

## $p_\text{T}$ smearing + jet rotation

In [9]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_{i:.1f}_aug_1_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_pT_jet_aug_1_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, pT smearing + Jet rotation: +1'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_0.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_pT_jet_aug_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, pT smearing + Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pT_jet_aug_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing + Jet rotation: +1"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_2.0_aug_1_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_pT_jet_aug_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, pT smearing + Jet rotation: +1"
python train_CNN.py ../Sample/HVmo

In [10]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_{i:.1f}_aug_3_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_25x25.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy'
    model_name = f'SB_{i:.1f}_pT_jet_aug_3_25x25'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 25x25, pT smearing + Jet rotation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_0.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_pT_jet_aug_3_25x25 "Sensitivity: 0.0, Resolution: 25x25, pT smearing + Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_1.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_pT_jet_aug_3_25x25 "Sensitivity: 1.0, Resolution: 25x25, pT smearing + Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/mix_sample_2.0_aug_3_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_pT_jet_aug_3_25x25 "Sensitivity: 2.0, Resolution: 25x25, pT smearing + Jet rotation: +3"
python train_CNN.py ../Sample/HVmo

In [3]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_{i:.1f}_aug_3_75x75.npy'
    val_file = f'../Sample/HVmodel/data/split_val/mix_sample_{i:.1f}_val_75x75.npy'
    true_label_file = '../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy'
    model_name = f'SB_{i:.1f}_pT_jet_aug_3_75x75'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: 75x75, pT smearing + Jet rotation: +3'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_0.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_0.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_0.0_pT_jet_aug_3_75x75 "Sensitivity: 0.0, Resolution: 75x75, pT smearing + Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_1.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_1.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_1.0_pT_jet_aug_3_75x75 "Sensitivity: 1.0, Resolution: 75x75, pT smearing + Jet rotation: +3"
python train_CNN.py ../Sample/HVmodel/data/pt_smearing_jet_rotation/75x75/mix_sample_2.0_aug_3_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_2.0_val_75x75.npy ../Sample/HVmodel/data/split_val/mix_sample_test_75x75.npy SB_2.0_pT_jet_aug_3_75x75 "Sensitivity: 2.0, Resolution: 75x75, pT smearing + Jet rotation: +3"
python train_CNN.py ../Sample/HVmo

## Different training dataset

In [5]:
seeds = [i * 100 + 23 for i in range(1, 10)]
for seed in seeds:
    res = 25

    for i in range(11):
        train_file = f'../Sample/HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_{res}x{res}.npy'
        val_file = f'../Sample/HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
        true_label_file = f'../Sample/HVmodel/data/split_val/mix_sample_test_{res}x{res}.npy'
        model_name = f'SB_{i:.1f}_{res}x{res}_sn_{seed}'
        sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Random Seed: {seed}'
        cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
        print(cmd)

python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_0.0_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_25x25_sn_123 "Sensitivity: 0.0, Resolution: 25x25, Random Seed: 123"
python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_1.0_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_25x25_sn_123 "Sensitivity: 1.0, Resolution: 25x25, Random Seed: 123"
python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_2.0_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_25x25_sn_123 "Sensitivity: 2.0, Resolution: 25x25, Random Seed: 123"
python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_3.0_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample

In [7]:
seeds = [i * 100 + 23 for i in range(1, 10)]
for seed in seeds:
    res = 25
    n = 3
    for i in range(11):
        train_file = f'../Sample/HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_jet_aug_{n}_{res}x{res}.npy'
        val_file = f'../Sample/HVmodel/data/seed_{seed}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
        true_label_file = f'../Sample/HVmodel/data/split_val/mix_sample_test_{res}x{res}.npy'
        model_name = f'SB_{i:.1f}_jet_aug_{n}_{res}x{res}_sn_{seed}'
        sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +{n}, Random Seed: {seed}'
        cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
        print(cmd)

python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_0.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_0.0_jet_aug_3_25x25_sn_123 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +3, Random Seed: 123"
python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_1.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_1.0_jet_aug_3_25x25_sn_123 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +3, Random Seed: 123"
python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_2.0_jet_aug_3_25x25.npy ../Sample/HVmodel/data/seed_123/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/mix_sample_test_25x25.npy SB_2.0_jet_aug_3_25x25_sn_123 "Sensitivity: 2.0, Resolution: 25x25, Jet rotation: +3, Random Seed: 123"
python train_CNN.py ../Sample/HVmodel/data/seed_123/mix_sample_3.0_

## Hyperparameter optimization

In [4]:
res = 25
for i in range(11):
    train_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_{res}x{res}_best_hp'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_25x25_best_hp "Sensitivity: 0.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_25x25_best_hp "Sensitivity: 1.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_25x25_best_hp "Sensitivity: 2.0, Resolution: 25x25"
python train_CNN.py ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_3.0_val_25x25.npy ../Sample/HVmodel/da

In [2]:
res = 25
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/mix_sample_{i:.1f}_jet_aug_5_25x25.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_{res}x{res}_best_hp'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +5'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_sample_0.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_jet_aug_5_25x25_best_hp "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_sample_1.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_jet_aug_5_25x25_best_hp "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/mix_sample_2.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_jet_aug_5_25x25_best_hp "Sensitivity: 2.0, Resolution: 25x25, Jet rotation: +5"
python train_CNN.py ../Sample/HVmodel/data/jet_ro

## Jet rotation range

In [5]:
res = 25
angle = 90
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_range_{angle}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +5, Rotation range: {angle}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_90/mix_sample_0.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_jet_aug_5_range_90_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 90"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_90/mix_sample_1.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_jet_aug_5_range_90_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 90"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_90/mix_sample_2.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_jet_aug_5_range_90_25x25 "Sensitivity: 2.0, Resolution: 25x2

In [6]:
res = 25
angle = 5
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_range_{angle}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +5, Rotation range: {angle}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_5/mix_sample_0.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_jet_aug_5_range_5_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_5/mix_sample_1.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_jet_aug_5_range_5_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_5/mix_sample_2.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_jet_aug_5_range_5_25x25 "Sensitivity: 2.0, Resolution: 25x25, Jet r

In [1]:
res = 25
angle = 45
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_range_{angle}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +5, Rotation range: {angle}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_45/mix_sample_0.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_jet_aug_5_range_45_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 45"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_45/mix_sample_1.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_jet_aug_5_range_45_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 45"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_45/mix_sample_2.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_jet_aug_5_range_45_25x25 "Sensitivity: 2.0, Resolution: 25x2

In [1]:
res = 25
angle = 1
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/range_{angle}/mix_sample_{i:.1f}_jet_aug_5_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_5_range_{angle}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation: +5, Rotation range: {angle}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_1/mix_sample_0.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_jet_aug_5_range_1_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 1"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_1/mix_sample_1.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_jet_aug_5_range_1_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation: +5, Rotation range: 1"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/range_1/mix_sample_2.0_jet_aug_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_jet_aug_5_range_1_25x25 "Sensitivity: 2.0, Resolution: 25x25, Jet r

## Fixed rotation angles

In [5]:
res = 25
angles = '-5_5'
for i in range(11):
    train_file = f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/angles_{angles}/mix_sample_{i:.1f}_jet_aug_angles_{angles}_{res}x{res}.npy'
    val_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_{i:.1f}_val_{res}x{res}.npy'
    true_label_file = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
    model_name = f'SB_{i:.1f}_jet_aug_angles_{angles}_{res}x{res}'
    sample_type = f'Sensitivity: {i:.1f}, Resolution: {res}x{res}, Jet rotation, Rotation angles: {angles}'
    cmd = f'python train_CNN.py {train_file} {val_file} {true_label_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_0.0_jet_aug_angles_-5_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_0.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_0.0_jet_aug_angles_-5_5_25x25 "Sensitivity: 0.0, Resolution: 25x25, Jet rotation, Rotation angles: -5_5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_1.0_jet_aug_angles_-5_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_1.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_1.0_jet_aug_angles_-5_5_25x25 "Sensitivity: 1.0, Resolution: 25x25, Jet rotation, Rotation angles: -5_5"
python train_CNN.py ../Sample/HVmodel/data/jet_rotation/25x25/angles_-5_5/mix_sample_2.0_jet_aug_angles_-5_5_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_2.0_val_25x25.npy ../Sample/HVmodel/data/split_val/25x25/mix_sample_test_25x25.npy SB_2.0_jet_aug_angles_-5_5_2

# CNN Training with Configuration file

## Origin

In [2]:
i = 1
res = 25
for seed in range(117, 1100, 20):
    for s in range(11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-2.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

for seed in range(103, 1100, 20):
    for s in [*range(5, 11), *range(5)]:
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-2.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/origin_25x25_config_01.json
python train_CNN.py config_files/origin_25x25_config_02.json
python train_CNN.py config_files/origin_25x25_config_03.json
python train_CNN.py config_files/origin_25x25_config_04.json
python train_CNN.py config_files/origin_25x25_config_05.json
python train_CNN.py config_files/origin_25x25_config_06.json
python train_CNN.py config_files/origin_25x25_config_07.json
python train_CNN.py config_files/origin_25x25_config_08.json
python train_CNN.py config_files/origin_25x25_config_09.json
python train_CNN.py config_files/origin_25x25_config_10.json
python train_CNN.py config_files/origin_25x25_config_11.json
python train_CNN.py config_files/origin_25x25_config_12.json
python train_CNN.py config_files/origin_25x25_config_13.json
python train_CNN.py config_files/origin_25x25_config_14.json
python train_CNN.py config_files/origin_25x25_config_15.json
python train_CNN.py config_files/origin_25x25_config_16.json
python train_CNN.py conf

In [2]:
i = 1
res = 25
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_dp_035_CNN_l1_00001'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

for seed in range(123, 1100, 100):
    for s in range(6, 11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_dp_035_CNN_l1_00001'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/origin_25x25_config_01.json
python train_CNN.py config_files/origin_25x25_config_02.json
python train_CNN.py config_files/origin_25x25_config_03.json
python train_CNN.py config_files/origin_25x25_config_04.json
python train_CNN.py config_files/origin_25x25_config_05.json
python train_CNN.py config_files/origin_25x25_config_06.json
python train_CNN.py config_files/origin_25x25_config_07.json
python train_CNN.py config_files/origin_25x25_config_08.json
python train_CNN.py config_files/origin_25x25_config_09.json
python train_CNN.py config_files/origin_25x25_config_10.json
python train_CNN.py config_files/origin_25x25_config_11.json
python train_CNN.py config_files/origin_25x25_config_12.json
python train_CNN.py config_files/origin_25x25_config_13.json
python train_CNN.py config_files/origin_25x25_config_14.json
python train_CNN.py config_files/origin_25x25_config_15.json
python train_CNN.py config_files/origin_25x25_config_16.json
python train_CNN.py conf

In [3]:
i = 1
res = 25
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_5_fold'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_k-fold.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_k-fold.py config_files/origin_25x25_config_01.json
python train_CNN_k-fold.py config_files/origin_25x25_config_02.json
python train_CNN_k-fold.py config_files/origin_25x25_config_03.json
python train_CNN_k-fold.py config_files/origin_25x25_config_04.json
python train_CNN_k-fold.py config_files/origin_25x25_config_05.json
python train_CNN_k-fold.py config_files/origin_25x25_config_06.json
python train_CNN_k-fold.py config_files/origin_25x25_config_07.json
python train_CNN_k-fold.py config_files/origin_25x25_config_08.json
python train_CNN_k-fold.py config_files/origin_25x25_config_09.json
python train_CNN_k-fold.py config_files/origin_25x25_config_10.json
python train_CNN_k-fold.py config_files/origin_25x25_config_11.json
python train_CNN_k-fold.py config_files/origin_25x25_config_12.json
python train_CNN_k-fold.py config_files/origin_25x25_config_13.json
python train_CNN_k-fold.py config_files/origin_25x25_config_14.json
python train_CNN_k-fold.py config_files/origin_2

## Jet rotation

In [2]:
i = 1
res = 25
aug = 3
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_dp_045'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/jet_aug_3_25x25_config_01.json
python train_CNN.py config_files/jet_aug_3_25x25_config_02.json
python train_CNN.py config_files/jet_aug_3_25x25_config_03.json
python train_CNN.py config_files/jet_aug_3_25x25_config_04.json
python train_CNN.py config_files/jet_aug_3_25x25_config_05.json
python train_CNN.py config_files/jet_aug_3_25x25_config_06.json
python train_CNN.py config_files/jet_aug_3_25x25_config_07.json
python train_CNN.py config_files/jet_aug_3_25x25_config_08.json
python train_CNN.py config_files/jet_aug_3_25x25_config_09.json
python train_CNN.py config_files/jet_aug_3_25x25_config_10.json
python train_CNN.py config_files/jet_aug_3_25x25_config_11.json
python train_CNN.py config_files/jet_aug_3_25x25_config_12.json
python train_CNN.py config_files/jet_aug_3_25x25_config_13.json
python train_CNN.py config_files/jet_aug_3_25x25_config_14.json
python train_CNN.py config_files/jet_aug_3_25x25_config_15.json
python train_CNN.py config_files/jet_aug

In [5]:
i = 1
res = 25
s = 0
for seed in range(123, 1100, 100):
    for aug in [5, 7, 10, 15]:
        config_path = f'config_files/jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/jet_aug_5_25x25_config_01.json
python train_CNN.py config_files/jet_aug_7_25x25_config_02.json
python train_CNN.py config_files/jet_aug_10_25x25_config_03.json
python train_CNN.py config_files/jet_aug_15_25x25_config_04.json
python train_CNN.py config_files/jet_aug_5_25x25_config_05.json
python train_CNN.py config_files/jet_aug_7_25x25_config_06.json
python train_CNN.py config_files/jet_aug_10_25x25_config_07.json
python train_CNN.py config_files/jet_aug_15_25x25_config_08.json
python train_CNN.py config_files/jet_aug_5_25x25_config_09.json
python train_CNN.py config_files/jet_aug_7_25x25_config_10.json
python train_CNN.py config_files/jet_aug_10_25x25_config_11.json
python train_CNN.py config_files/jet_aug_15_25x25_config_12.json
python train_CNN.py config_files/jet_aug_5_25x25_config_13.json
python train_CNN.py config_files/jet_aug_7_25x25_config_14.json
python train_CNN.py config_files/jet_aug_10_25x25_config_15.json
python train_CNN.py config_files/

## $p_\text{T}$ smearing + Jet rotation

In [3]:
i = 1
res = 25
aug = 3
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_dp_045'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_jet_aug_3_25x25_config_01.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_02.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_03.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_04.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_05.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_06.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_07.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_08.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_09.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_10.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_11.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_12.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_13.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_14.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_15.

In [6]:
i = 1
res = 25
s = 0
for seed in range(123, 1100, 100):
    for aug in [5, 7, 10, 15]:
        config_path = f'config_files/pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_jet_aug_5_25x25_config_01.json
python train_CNN.py config_files/pt_jet_aug_7_25x25_config_02.json
python train_CNN.py config_files/pt_jet_aug_10_25x25_config_03.json
python train_CNN.py config_files/pt_jet_aug_15_25x25_config_04.json
python train_CNN.py config_files/pt_jet_aug_5_25x25_config_05.json
python train_CNN.py config_files/pt_jet_aug_7_25x25_config_06.json
python train_CNN.py config_files/pt_jet_aug_10_25x25_config_07.json
python train_CNN.py config_files/pt_jet_aug_15_25x25_config_08.json
python train_CNN.py config_files/pt_jet_aug_5_25x25_config_09.json
python train_CNN.py config_files/pt_jet_aug_7_25x25_config_10.json
python train_CNN.py config_files/pt_jet_aug_10_25x25_config_11.json
python train_CNN.py config_files/pt_jet_aug_15_25x25_config_12.json
python train_CNN.py config_files/pt_jet_aug_5_25x25_config_13.json
python train_CNN.py config_files/pt_jet_aug_7_25x25_config_14.json
python train_CNN.py config_files/pt_jet_aug_10_25x25_con

In [9]:
i = 1
res = 25
aug = 5
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/pt_jet_aug_{aug}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_jet_aug_5_config_01.json
python train_CNN.py config_files/pt_jet_aug_5_config_02.json
python train_CNN.py config_files/pt_jet_aug_5_config_03.json
python train_CNN.py config_files/pt_jet_aug_5_config_04.json
python train_CNN.py config_files/pt_jet_aug_5_config_05.json
python train_CNN.py config_files/pt_jet_aug_5_config_06.json
python train_CNN.py config_files/pt_jet_aug_5_config_07.json
python train_CNN.py config_files/pt_jet_aug_5_config_08.json
python train_CNN.py config_files/pt_jet_aug_5_config_09.json
python train_CNN.py config_files/pt_jet_aug_5_config_10.json
python train_CNN.py config_files/pt_jet_aug_5_config_11.json
python train_CNN.py config_files/pt_jet_aug_5_config_12.json
python train_CNN.py config_files/pt_jet_aug_5_config_13.json
python train_CNN.py config_files/pt_jet_aug_5_config_14.json
python train_CNN.py config_files/pt_jet_aug_5_config_15.json
python train_CNN.py config_files/pt_jet_aug_5_config_16.json
python train_CNN.py conf

## $\eta-\phi$ smeearing + $p_\text{T}$ smearing + Jet rotation

In [8]:
i = 1
res = 25
aug = 3
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/eta_phi_pt_jet_aug_{aug}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/eta_phi_pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
            data['model_name'] = f'SB_{s:.1f}_eta_phi_pt_jet_aug_{aug}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Eta Phi pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_01.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_02.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_03.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_04.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_05.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_06.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_07.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_08.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_09.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_10.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_11.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_12.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_13.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_3_config_14.json
python train_CNN.py config_files/e

In [7]:
i = 1
res = 25
aug = 5
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/eta_phi_pt_jet_aug_{aug}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/eta_phi_pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
            data['model_name'] = f'SB_{s:.1f}_eta_phi_pt_jet_aug_{aug}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Eta Phi pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_01.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_02.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_03.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_04.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_05.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_06.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_07.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_08.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_09.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_10.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_11.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_12.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_13.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_5_config_14.json
python train_CNN.py config_files/e

## Asymptotic behaviour

In [3]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [7, 10, 15, 22, 33]:
        config_path = f'config_files/jet_aug_{aug}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/jet_aug_7_config_01.json
python train_CNN.py config_files/jet_aug_10_config_02.json
python train_CNN.py config_files/jet_aug_15_config_03.json
python train_CNN.py config_files/jet_aug_22_config_04.json
python train_CNN.py config_files/jet_aug_33_config_05.json
python train_CNN.py config_files/jet_aug_7_config_06.json
python train_CNN.py config_files/jet_aug_10_config_07.json
python train_CNN.py config_files/jet_aug_15_config_08.json
python train_CNN.py config_files/jet_aug_22_config_09.json
python train_CNN.py config_files/jet_aug_33_config_10.json
python train_CNN.py config_files/jet_aug_7_config_11.json
python train_CNN.py config_files/jet_aug_10_config_12.json
python train_CNN.py config_files/jet_aug_15_config_13.json
python train_CNN.py config_files/jet_aug_22_config_14.json
python train_CNN.py config_files/jet_aug_33_config_15.json
python train_CNN.py config_files/jet_aug_7_config_16.json
python train_CNN.py config_files/jet_aug_10_config_17.json
p

In [4]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [33]:
        config_path = f'config_files/pt_jet_aug_{aug}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_jet_aug_33_config_01.json
python train_CNN.py config_files/pt_jet_aug_33_config_02.json
python train_CNN.py config_files/pt_jet_aug_33_config_03.json
python train_CNN.py config_files/pt_jet_aug_33_config_04.json
python train_CNN.py config_files/pt_jet_aug_33_config_05.json
python train_CNN.py config_files/pt_jet_aug_33_config_06.json
python train_CNN.py config_files/pt_jet_aug_33_config_07.json
python train_CNN.py config_files/pt_jet_aug_33_config_08.json
python train_CNN.py config_files/pt_jet_aug_33_config_09.json
python train_CNN.py config_files/pt_jet_aug_33_config_10.json


In [5]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [33]:
        config_path = f'config_files/eta_phi_pt_jet_aug_{aug}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/eta_phi_pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test_{res}x{res}.npy'
            data['model_name'] = f'SB_{s:.1f}_eta_phi_pt_jet_aug_{aug}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Eta Phi pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_01.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_02.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_03.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_04.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_05.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_06.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_07.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_08.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_09.json
python train_CNN.py config_files/eta_phi_pt_jet_aug_33_config_10.json


## Shift to $p_\text{T}$ center

In [6]:
i = 1
res = 25
l = 5
for seed in range(123, 1100, 100):
    for s in range(6):
        config_path = f'config_files/pt_center_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25']
            data['val_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139 * l
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_center_x{l}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Luminosity: x{l}, Resolution: {res}x{res}, Seed: {seed}, Only shift to pT center'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

for seed in range(123, 1100, 100):
    for s in range(6, 11):
        config_path = f'config_files/pt_center_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25']
            data['val_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139 * l
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_center_x{l}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Luminosity: x{l}, Resolution: {res}x{res}, Seed: {seed}, Only shift to pT center'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_center_config_01.json
python train_CNN.py config_files/pt_center_config_02.json
python train_CNN.py config_files/pt_center_config_03.json
python train_CNN.py config_files/pt_center_config_04.json
python train_CNN.py config_files/pt_center_config_05.json
python train_CNN.py config_files/pt_center_config_06.json
python train_CNN.py config_files/pt_center_config_07.json
python train_CNN.py config_files/pt_center_config_08.json
python train_CNN.py config_files/pt_center_config_09.json
python train_CNN.py config_files/pt_center_config_10.json
python train_CNN.py config_files/pt_center_config_11.json
python train_CNN.py config_files/pt_center_config_12.json
python train_CNN.py config_files/pt_center_config_13.json
python train_CNN.py config_files/pt_center_config_14.json
python train_CNN.py config_files/pt_center_config_15.json
python train_CNN.py config_files/pt_center_config_16.json
python train_CNN.py config_files/pt_center_config_17.json
python train_C

## Only augmentation

In [3]:
i = 1
res = 25
aug = 4
seed = 123
s = 0
config_path = f'config_files/only_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
with open(os.path.join('CNN', config_path), 'w') as f:
    data = {}

    data['train_npy_paths'] = [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
    data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
    data['seed'] = seed
    data['sensitivity'] = s
    data['luminosity'] = 139
    data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
    data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}'
    data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: {aug}, Seed: {seed}'

    json.dump(data, f, indent=4)

cmd = f'python train_CNN.py {config_path}'
print(cmd)

i += 1

config_path = f'config_files/only_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
with open(os.path.join('CNN', config_path), 'w') as f:
    data = {}

    data['train_npy_paths'] = [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
    data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
    data['seed'] = seed
    data['sensitivity'] = s
    data['luminosity'] = 139
    data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test.npy'
    data['model_name'] = f'SB_{s:.1f}_only_pt_jet_aug_{aug}_{res}x{res}'
    data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: {aug}, Seed: {seed}'

    json.dump(data, f, indent=4)

cmd = f'python train_CNN.py {config_path}'
print(cmd)

i += 1

python train_CNN.py config_files/only_jet_aug_4_25x25_config_01.json
python train_CNN.py config_files/only_pt_jet_aug_4_25x25_config_02.json


## Sensitivity = 0

In [2]:
i = 1
res = 25
s = 0
for seed in range(10):

    config_path = f'config_files/FY0_config_{i:02}.json'
    with open(os.path.join('CNN', config_path), 'w') as f:
        data = {}

        # data['train_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25']
        data['train_npy_path'] = f'../Sample/HVmodel/data/FY-sig-0/seed-{seed}-tr.npy'
        # data['val_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25/val']
        data['val_npy_path'] = f'../Sample/HVmodel/data/FY-sig-0/seed-{seed}-val.npy'
        data['seed'] = seed
        data['sensitivity'] = s
        data['luminosity'] = 139
        data['true_label_path'] = f'../Sample/HVmodel/data/FY-sig-0/seed-{seed}-te.npy'
        data['model_name'] = f'SB_{s:.1f}_25x25_FY'
        data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, FY'

        json.dump(data, f, indent=4)

    cmd = f'python train_CNN2.py {config_path}'
    print(cmd)

    i += 1

i = 1
res = 25
s = 0
for seed in range(10):

    config_path = f'config_files/ZN0_config_{i:02}.json'
    with open(os.path.join('CNN', config_path), 'w') as f:
        data = {}

        # data['train_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25']
        data['train_npy_path'] = f'../Sample/HVmodel/data/ZN-sig-0/seed-{seed}-tr.npy'
        # data['val_npy_paths'] = ['../Sample/HVmodel/data/pt_center/25x25/val']
        data['val_npy_path'] = f'../Sample/HVmodel/data/ZN-sig-0/seed-{seed}-val.npy'
        data['seed'] = seed
        data['sensitivity'] = s
        data['luminosity'] = 139
        data['true_label_path'] = f'../Sample/HVmodel/data/ZN-sig-0/seed-{seed}-te.npy'
        data['model_name'] = f'SB_{s:.1f}_25x25_ZN'
        data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, ZN'

        json.dump(data, f, indent=4)

    cmd = f'python train_CNN2.py {config_path}'
    print(cmd)

    i += 1

python train_CNN2.py config_files/FY0_config_01.json
python train_CNN2.py config_files/FY0_config_02.json
python train_CNN2.py config_files/FY0_config_03.json
python train_CNN2.py config_files/FY0_config_04.json
python train_CNN2.py config_files/FY0_config_05.json
python train_CNN2.py config_files/FY0_config_06.json
python train_CNN2.py config_files/FY0_config_07.json
python train_CNN2.py config_files/FY0_config_08.json
python train_CNN2.py config_files/FY0_config_09.json
python train_CNN2.py config_files/FY0_config_10.json
python train_CNN2.py config_files/ZN0_config_01.json
python train_CNN2.py config_files/ZN0_config_02.json
python train_CNN2.py config_files/ZN0_config_03.json
python train_CNN2.py config_files/ZN0_config_04.json
python train_CNN2.py config_files/ZN0_config_05.json
python train_CNN2.py config_files/ZN0_config_06.json
python train_CNN2.py config_files/ZN0_config_07.json
python train_CNN2.py config_files/ZN0_config_08.json
python train_CNN2.py config_files/ZN0_config_0

## Another pool

In [3]:
i = 1
res = 25
for seed in range(10):
    for s in range(11):
        config_path = f'config_files/ZN_pool_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = ['../Sample/HVmodel/data/ZN-pool/train']
            data['val_npy_paths'] = ['../Sample/HVmodel/data/ZN-pool/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-2.npy'
            data['model_name'] = f'SB_{s:.1f}_25x25_ZN_pool'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, ZN-pool'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1


python train_CNN.py config_files/ZN_pool_config_01.json
python train_CNN.py config_files/ZN_pool_config_02.json
python train_CNN.py config_files/ZN_pool_config_03.json
python train_CNN.py config_files/ZN_pool_config_04.json
python train_CNN.py config_files/ZN_pool_config_05.json
python train_CNN.py config_files/ZN_pool_config_06.json
python train_CNN.py config_files/ZN_pool_config_07.json
python train_CNN.py config_files/ZN_pool_config_08.json
python train_CNN.py config_files/ZN_pool_config_09.json
python train_CNN.py config_files/ZN_pool_config_10.json
python train_CNN.py config_files/ZN_pool_config_11.json
python train_CNN.py config_files/ZN_pool_config_12.json
python train_CNN.py config_files/ZN_pool_config_13.json
python train_CNN.py config_files/ZN_pool_config_14.json
python train_CNN.py config_files/ZN_pool_config_15.json
python train_CNN.py config_files/ZN_pool_config_16.json
python train_CNN.py config_files/ZN_pool_config_17.json
python train_CNN.py config_files/ZN_pool_config_

## Another testing dataset

In [7]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-2.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN1.py config_files/origin_25x25_config_01.json
python train_CNN1.py config_files/origin_25x25_config_02.json
python train_CNN1.py config_files/origin_25x25_config_03.json
python train_CNN1.py config_files/origin_25x25_config_04.json
python train_CNN1.py config_files/origin_25x25_config_05.json
python train_CNN1.py config_files/origin_25x25_config_06.json
python train_CNN1.py config_files/origin_25x25_config_07.json
python train_CNN1.py config_files/origin_25x25_config_08.json
python train_CNN1.py config_files/origin_25x25_config_09.json
python train_CNN1.py config_files/origin_25x25_config_10.json
python train_CNN1.py config_files/origin_25x25_config_11.json
python train_CNN1.py config_files/origin_25x25_config_12.json
python train_CNN1.py config_files/origin_25x25_config_13.json
python train_CNN1.py config_files/origin_25x25_config_14.json
python train_CNN1.py config_files/origin_25x25_config_15.json
python train_CNN1.py config_files/origin_25x25_config_16.json
python t

In [3]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-3.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN1.py config_files/origin_25x25_config_01.json
python train_CNN1.py config_files/origin_25x25_config_02.json
python train_CNN1.py config_files/origin_25x25_config_03.json
python train_CNN1.py config_files/origin_25x25_config_04.json
python train_CNN1.py config_files/origin_25x25_config_05.json
python train_CNN1.py config_files/origin_25x25_config_06.json
python train_CNN1.py config_files/origin_25x25_config_07.json
python train_CNN1.py config_files/origin_25x25_config_08.json
python train_CNN1.py config_files/origin_25x25_config_09.json
python train_CNN1.py config_files/origin_25x25_config_10.json
python train_CNN1.py config_files/origin_25x25_config_11.json
python train_CNN1.py config_files/origin_25x25_config_12.json
python train_CNN1.py config_files/origin_25x25_config_13.json
python train_CNN1.py config_files/origin_25x25_config_14.json
python train_CNN1.py config_files/origin_25x25_config_15.json
python train_CNN1.py config_files/origin_25x25_config_16.json
python t

## Larger testing set

In [6]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_test4'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN1.py config_files/origin_25x25_config_01.json
python train_CNN1.py config_files/origin_25x25_config_02.json
python train_CNN1.py config_files/origin_25x25_config_03.json
python train_CNN1.py config_files/origin_25x25_config_04.json
python train_CNN1.py config_files/origin_25x25_config_05.json
python train_CNN1.py config_files/origin_25x25_config_06.json
python train_CNN1.py config_files/origin_25x25_config_07.json
python train_CNN1.py config_files/origin_25x25_config_08.json
python train_CNN1.py config_files/origin_25x25_config_09.json
python train_CNN1.py config_files/origin_25x25_config_10.json
python train_CNN1.py config_files/origin_25x25_config_11.json
python train_CNN1.py config_files/origin_25x25_config_12.json
python train_CNN1.py config_files/origin_25x25_config_13.json
python train_CNN1.py config_files/origin_25x25_config_14.json
python train_CNN1.py config_files/origin_25x25_config_15.json
python train_CNN1.py config_files/origin_25x25_config_16.json
python t

In [2]:
i = 1
res = 25
for seed in range(10):
    for s in range(11):
        config_path = f'config_files/ZN_pool_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = ['../Sample/HVmodel/data/ZN-pool/train']
            data['val_npy_paths'] = ['../Sample/HVmodel/data/ZN-pool/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_ZN_pool_test4'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, ZN-pool'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/ZN_pool_config_01.json
python train_CNN.py config_files/ZN_pool_config_02.json
python train_CNN.py config_files/ZN_pool_config_03.json
python train_CNN.py config_files/ZN_pool_config_04.json
python train_CNN.py config_files/ZN_pool_config_05.json
python train_CNN.py config_files/ZN_pool_config_06.json
python train_CNN.py config_files/ZN_pool_config_07.json
python train_CNN.py config_files/ZN_pool_config_08.json
python train_CNN.py config_files/ZN_pool_config_09.json
python train_CNN.py config_files/ZN_pool_config_10.json
python train_CNN.py config_files/ZN_pool_config_11.json
python train_CNN.py config_files/ZN_pool_config_12.json
python train_CNN.py config_files/ZN_pool_config_13.json
python train_CNN.py config_files/ZN_pool_config_14.json
python train_CNN.py config_files/ZN_pool_config_15.json
python train_CNN.py config_files/ZN_pool_config_16.json
python train_CNN.py config_files/ZN_pool_config_17.json
python train_CNN.py config_files/ZN_pool_config_

In [3]:
i = 1
res = 25
aug = 3
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_test4'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_jet_aug_3_25x25_config_01.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_02.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_03.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_04.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_05.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_06.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_07.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_08.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_09.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_10.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_11.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_12.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_13.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_14.json
python train_CNN.py config_files/pt_jet_aug_3_25x25_config_15.

## $m_{jj}$ normalization

In [2]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/pt_normalize_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/pt_normalization/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/pt_normalization/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/pt_normalization/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_pt_nor'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN.py {config_path}'
        print(cmd)

        i += 1

python train_CNN.py config_files/pt_normalize_25x25_config_01.json
python train_CNN.py config_files/pt_normalize_25x25_config_02.json
python train_CNN.py config_files/pt_normalize_25x25_config_03.json
python train_CNN.py config_files/pt_normalize_25x25_config_04.json
python train_CNN.py config_files/pt_normalize_25x25_config_05.json
python train_CNN.py config_files/pt_normalize_25x25_config_06.json
python train_CNN.py config_files/pt_normalize_25x25_config_07.json
python train_CNN.py config_files/pt_normalize_25x25_config_08.json
python train_CNN.py config_files/pt_normalize_25x25_config_09.json
python train_CNN.py config_files/pt_normalize_25x25_config_10.json
python train_CNN.py config_files/pt_normalize_25x25_config_11.json
python train_CNN.py config_files/pt_normalize_25x25_config_12.json
python train_CNN.py config_files/pt_normalize_25x25_config_13.json
python train_CNN.py config_files/pt_normalize_25x25_config_14.json
python train_CNN.py config_files/pt_normalize_25x25_config_15.

## SRSB normalization

In [2]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/SRSB_normalize_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_SRSB_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, SRSB normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_SRSB_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_01.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_02.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_03.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_04.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_05.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_06.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_07.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_08.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_09.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_10.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_11.json
python train_CNN_SRSB_norm.py config_files/SRSB_normalize_25x25_config_12.json
python train_CNN_SRSB_norm.py config_files/SRSB_norm

## $p_{\text{T}}$ normalization

### Image normalization

In [4]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/image_pT_normalize_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_01.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_02.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_03.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_04.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_05.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_06.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_07.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_08.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_09.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_10.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_11.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_25x25_config_12.json
python train_CNN

### Event normalization

In [2]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_ZN_pool_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/train']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_event_pT_norm_ZN_pool'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, event pT normalization, ZN-pool'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_01.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_02.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_03.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_04.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_05.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_06.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_07.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_08.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_09.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_10.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_ZN_pool_25x25_config_11.json
python train_CNN_pT_n

In [3]:
i = 1
res = 25
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_normalize_25x25_config_01.json"
"config_files/event_pT_normalize_25x25_config_02.json"
"config_files/event_pT_normalize_25x25_config_03.json"
"config_files/event_pT_normalize_25x25_config_04.json"
"config_files/event_pT_normalize_25x25_config_05.json"
"config_files/event_pT_normalize_25x25_config_06.json"
"config_files/event_pT_normalize_25x25_config_07.json"
"config_files/event_pT_normalize_25x25_config_08.json"
"config_files/event_pT_normalize_25x25_config_09.json"
"config_files/event_pT_normalize_25x25_config_10.json"
"config_files/event_pT_normalize_25x25_config_11.json"
"config_files/event_pT_normalize_25x25_config_12.json"
"config_files/event_pT_normalize_25x25_config_13.json"
"config_files/event_pT_normalize_25x25_config_14.json"
"config_files/event_pT_normalize_25x25_config_15.json"
"config_files/event_pT_normalize_25x25_config_16.json"
"config_files/event_pT_normalize_25x25_config_17.json"
"config_files/event_pT_normalize_25x25_config_18.json"
"config_fi

In [2]:
i = 1
res = 25
aug = 3
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_01.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_02.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_03.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_04.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_05.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_06.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_07.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_08.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_09.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_10.json
python train_CNN_pT_norm.py config_files/event_pT_normalize_jet_aug_3_25x25_config_11.json

In [3]:
i = 1
res = 25
aug = 3
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_01.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_02.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_03.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_04.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_05.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_06.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_07.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_08.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_09.json
python train_CNN_pT_norm1.py config_files/event_pT_normalize_pt_jet_aug_3_25x25_config_10.json
python train_CNN_pT_norm1.py config_files/event_pT

### ZN samples

In [3]:
i = 1
res = 25

for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/image_pT_normalize_ZN_pool_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/train']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_image_pT_norm_ZN_pool'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, image pT normalization, ZN-pool'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_01.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_02.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_03.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_04.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_05.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_06.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_07.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_08.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_09.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_10.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_ZN_pool_25x25_config_11.json
python train_CNN_pT_n

In [5]:
i = 1
res = 25
s = 0
for seed in range(1, 11):

    config_path = f'config_files/image_pT_normalize_ZN_data_{res}x{res}_config_{i:02}.json'
    with open(os.path.join('CNN', config_path), 'w') as f:
        data = {}
        # data['train_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/train']
        data['train_npy_path'] = f'../Sample/HVmodel/data/ZN-data/wo-imagenorm-sigma-0/data-{seed}/tr.npy'
        # data['val_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/val']
        data['val_npy_path'] = f'../Sample/HVmodel/data/ZN-data/wo-imagenorm-sigma-0/data-{seed}/vl.npy'
        data['ns'] = [0, 0, 20000, 21014]
        data['seed'] = seed
        data['sensitivity'] = s
        data['luminosity'] = 139
        # data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
        data['true_label_path'] = f'../Sample/HVmodel/data/ZN-data/wo-imagenorm-sigma-0/data-{seed}/te.npy'
        data['model_name'] = f'SB_{s:.1f}_{res}x{res}_image_norm_ZN_data'
        data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, image normalization, ZN data'

        json.dump(data, f, indent=4)

    cmd = f'python train_CNN_pT_norm2.py {config_path}'
    print(cmd)

    i += 1

s = 7.071
for seed in range(1, 11):

    config_path = f'config_files/image_pT_normalize_ZN_data_{res}x{res}_config_{i:02}.json'
    with open(os.path.join('CNN', config_path), 'w') as f:
        data = {}
        # data['train_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/train']
        data['train_npy_path'] = f'../Sample/HVmodel/data/ZN-data/wo-imagenorm-sigma-7/data-{seed}/tr.npy'
        # data['val_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/val']
        data['val_npy_path'] = f'../Sample/HVmodel/data/ZN-data/wo-imagenorm-sigma-7/data-{seed}/vl.npy'
        data['ns'] = [1000, 244, 20000, 21014]
        data['seed'] = seed
        data['sensitivity'] = s
        data['luminosity'] = 139
        # data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
        data['true_label_path'] = f'../Sample/HVmodel/data/ZN-data/wo-imagenorm-sigma-7/data-{seed}/te.npy'
        data['model_name'] = f'SB_{s:.1f}_{res}x{res}_image_norm_ZN_data'
        data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, image normalization, ZN data'

        json.dump(data, f, indent=4)

    cmd = f'python train_CNN_pT_norm2.py {config_path}'
    print(cmd)

    i += 1

s = 0
for seed in range(1, 11):

    config_path = f'config_files/image_pT_normalize_ZN_data_{res}x{res}_config_{i:02}.json'
    with open(os.path.join('CNN', config_path), 'w') as f:
        data = {}
        # data['train_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/train']
        data['train_npy_path'] = f'../Sample/HVmodel/data/ZN-data/w-imagenorm-sigma-0/data-{seed}/tr.npy'
        # data['val_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/val']
        data['val_npy_path'] = f'../Sample/HVmodel/data/ZN-data/w-imagenorm-sigma-0/data-{seed}/vl.npy'
        data['ns'] = [0, 0, 20000, 21014]
        data['seed'] = seed
        data['sensitivity'] = s
        data['luminosity'] = 139
        # data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
        data['true_label_path'] = f'../Sample/HVmodel/data/ZN-data/w-imagenorm-sigma-0/data-{seed}/te.npy'
        data['model_name'] = f'SB_{s:.1f}_{res}x{res}_ZN_image_norm_data'
        data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, ZN image normalization data'

        json.dump(data, f, indent=4)

    cmd = f'python train_CNN_pT_norm3.py {config_path}'
    print(cmd)

    i += 1

s = 7.071
for seed in range(1, 11):

    config_path = f'config_files/image_pT_normalize_ZN_data_{res}x{res}_config_{i:02}.json'
    with open(os.path.join('CNN', config_path), 'w') as f:
        data = {}
        # data['train_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/train']
        data['train_npy_path'] = f'../Sample/HVmodel/data/ZN-data/w-imagenorm-sigma-7/data-{seed}/tr.npy'
        # data['val_npy_paths'] = [f'../Sample/HVmodel/data/ZN-pool/res25/og/val']
        data['val_npy_path'] = f'../Sample/HVmodel/data/ZN-data/w-imagenorm-sigma-7/data-{seed}/vl.npy'
        data['ns'] = [1000, 244, 20000, 21014]
        data['seed'] = seed
        data['sensitivity'] = s
        data['luminosity'] = 139
        # data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
        data['true_label_path'] = f'../Sample/HVmodel/data/ZN-data/w-imagenorm-sigma-7/data-{seed}/te.npy'
        data['model_name'] = f'SB_{s:.1f}_{res}x{res}_ZN_image_norm_data'
        data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, ZN image normalization data'

        json.dump(data, f, indent=4)

    cmd = f'python train_CNN_pT_norm3.py {config_path}'
    print(cmd)

    i += 1

python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_01.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_02.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_03.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_04.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_05.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_06.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_07.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_08.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_09.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_10.json
python train_CNN_pT_norm2.py config_files/image_pT_normalize_ZN_data_25x25_config_11.json
python tra

## Data augmentation with image normalization

In [ ]:
i = 1
res = 25
aug = 5
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/image_pT_normalize_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_01.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_02.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_03.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_04.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_05.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_06.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_07.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_08.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_09.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_10.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_5_25x25_config_11.json

In [ ]:
i = 1
res = 25
aug = 10
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/image_pT_normalize_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_01.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_02.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_03.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_04.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_05.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_06.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_07.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_08.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_09.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_config_10.json
python train_CNN_pT_norm.py config_files/image_pT_normalize_jet_aug_10_25x25_con

In [ ]:
i = 1
res = 25
aug = 5
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/image_pT_normalize_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_01.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_02.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_03.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_04.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_05.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_06.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_07.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_08.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_09.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_5_25x25_config_10.json
python train_CNN_pT_norm1.py config_files/image_pT

In [ ]:
i = 1
res = 25
aug = 10
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/image_pT_normalize_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_01.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_02.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_03.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_04.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_05.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_06.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_07.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_08.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_09.json
python train_CNN_pT_norm1.py config_files/image_pT_normalize_pt_jet_aug_10_25x25_config_10.json
python train_CNN_pT_norm1.py config_file

### Asympototic behaviour

In [2]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [7, 15, 22, 33]:
        config_path = f'config_files/image_pT_norm_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_7_25x25_config_01.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_15_25x25_config_02.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_22_25x25_config_03.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_33_25x25_config_04.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_7_25x25_config_05.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_15_25x25_config_06.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_22_25x25_config_07.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_33_25x25_config_08.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_7_25x25_config_09.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_15_25x25_config_10.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_22_25x25_config_11.json
python train_CNN_pT_norm.py config_files/image

In [3]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [7, 15, 22, 33]:
        config_path = f'config_files/image_pT_norm_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_7_25x25_config_01.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_15_25x25_config_02.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_22_25x25_config_03.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_33_25x25_config_04.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_7_25x25_config_05.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_15_25x25_config_06.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_22_25x25_config_07.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_33_25x25_config_08.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_7_25x25_config_09.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_15_25x25_config_10.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_22_25x25_config_11.json
py

In [3]:
i = 1
res = 25
s = 5.0
for seed in range(123, 1100, 100):
    for aug in [7, 15, 22, 33]:
        config_path = f'config_files/image_pT_norm_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_7_25x25_config_01.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_15_25x25_config_02.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_22_25x25_config_03.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_33_25x25_config_04.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_7_25x25_config_05.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_15_25x25_config_06.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_22_25x25_config_07.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_33_25x25_config_08.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_7_25x25_config_09.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_15_25x25_config_10.json
python train_CNN_pT_norm.py config_files/image_pT_norm_jet_aug_22_25x25_config_11.json
python train_CNN_pT_norm.py config_files/image

In [3]:
i = 1
res = 25
s = 5.0
for seed in range(123, 1100, 100):
    for aug in [7, 15, 22, 33]:
        config_path = f'config_files/image_pT_norm_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_image_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, image pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm1.py {config_path}'
        print(cmd)

        i += 1

python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_7_25x25_config_01.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_15_25x25_config_02.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_22_25x25_config_03.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_33_25x25_config_04.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_7_25x25_config_05.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_15_25x25_config_06.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_22_25x25_config_07.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_33_25x25_config_08.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_7_25x25_config_09.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_15_25x25_config_10.json
python train_CNN_pT_norm1.py config_files/image_pT_norm_pt_jet_aug_22_25x25_config_11.json
py

## Data augmentation with event normalization

### +5, +10

In [5]:
i = 1
res = 25
aug = 5
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_normalize_jet_aug_5_25x25_config_01.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_02.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_03.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_04.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_05.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_06.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_07.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_08.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_09.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_10.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_11.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_12.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_13.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_14.json"
"config_files/event_pT_normalize_jet_aug_5_25x25_config_15.json"
"config_files/event_pT_no

In [3]:
i = 1
res = 25
aug = 10
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_normalize_jet_aug_10_25x25_config_01.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_02.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_03.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_04.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_05.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_06.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_07.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_08.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_09.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_10.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_11.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_12.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_13.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_14.json"
"config_files/event_pT_normalize_jet_aug_10_25x25_config_15.json"
"config_fi

In [4]:
i = 1
res = 25
aug = 5
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')


        i += 1

"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_01.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_02.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_03.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_04.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_05.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_06.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_07.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_08.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_09.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_10.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_11.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_12.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_13.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25x25_config_14.json"
"config_files/event_pT_normalize_pt_jet_aug_5_25

In [4]:
i = 1
res = 25
aug = 10
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')


        i += 1

"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_01.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_02.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_03.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_04.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_05.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_06.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_07.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_08.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_09.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_10.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_11.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_12.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_13.json"
"config_files/event_pT_normalize_pt_jet_aug_10_25x25_config_14.json"
"config_files/event_pT_normalize_p

### Asympototic behaviour

In [3]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [3, 7, 15, 22, 33]:
        config_path = f'config_files/event_pT_norm_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_norm_jet_aug_3_25x25_config_01.json"
"config_files/event_pT_norm_jet_aug_7_25x25_config_02.json"
"config_files/event_pT_norm_jet_aug_15_25x25_config_03.json"
"config_files/event_pT_norm_jet_aug_22_25x25_config_04.json"
"config_files/event_pT_norm_jet_aug_33_25x25_config_05.json"
"config_files/event_pT_norm_jet_aug_3_25x25_config_06.json"
"config_files/event_pT_norm_jet_aug_7_25x25_config_07.json"
"config_files/event_pT_norm_jet_aug_15_25x25_config_08.json"
"config_files/event_pT_norm_jet_aug_22_25x25_config_09.json"
"config_files/event_pT_norm_jet_aug_33_25x25_config_10.json"
"config_files/event_pT_norm_jet_aug_3_25x25_config_11.json"
"config_files/event_pT_norm_jet_aug_7_25x25_config_12.json"
"config_files/event_pT_norm_jet_aug_15_25x25_config_13.json"
"config_files/event_pT_norm_jet_aug_22_25x25_config_14.json"
"config_files/event_pT_norm_jet_aug_33_25x25_config_15.json"
"config_files/event_pT_norm_jet_aug_3_25x25_config_16.json"
"config_files/event_pT_norm_jet

In [4]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [3, 7, 15, 22, 33]:
        config_path = f'config_files/event_pT_norm_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_norm_pt_jet_aug_3_25x25_config_01.json"
"config_files/event_pT_norm_pt_jet_aug_7_25x25_config_02.json"
"config_files/event_pT_norm_pt_jet_aug_15_25x25_config_03.json"
"config_files/event_pT_norm_pt_jet_aug_22_25x25_config_04.json"
"config_files/event_pT_norm_pt_jet_aug_33_25x25_config_05.json"
"config_files/event_pT_norm_pt_jet_aug_3_25x25_config_06.json"
"config_files/event_pT_norm_pt_jet_aug_7_25x25_config_07.json"
"config_files/event_pT_norm_pt_jet_aug_15_25x25_config_08.json"
"config_files/event_pT_norm_pt_jet_aug_22_25x25_config_09.json"
"config_files/event_pT_norm_pt_jet_aug_33_25x25_config_10.json"
"config_files/event_pT_norm_pt_jet_aug_3_25x25_config_11.json"
"config_files/event_pT_norm_pt_jet_aug_7_25x25_config_12.json"
"config_files/event_pT_norm_pt_jet_aug_15_25x25_config_13.json"
"config_files/event_pT_norm_pt_jet_aug_22_25x25_config_14.json"
"config_files/event_pT_norm_pt_jet_aug_33_25x25_config_15.json"
"config_files/event_pT_norm_pt_jet_aug_3_25x25

In [4]:
i = 1
res = 25
s = 5.0
for seed in range(123, 1100, 100):
    for aug in [3, 7, 15, 22, 33]:
        config_path = f'config_files/event_pT_norm_SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_norm_SB_5.0_jet_aug_3_25x25_config_01.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_7_25x25_config_02.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_15_25x25_config_03.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_22_25x25_config_04.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_33_25x25_config_05.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_3_25x25_config_06.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_7_25x25_config_07.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_15_25x25_config_08.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_22_25x25_config_09.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_33_25x25_config_10.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_3_25x25_config_11.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_7_25x25_config_12.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_15_25x25_config_13.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_22_25x25_config_14.json"
"config_files/event_pT_norm_SB_5.0_jet_aug_33_25x25_co

In [5]:
i = 1
res = 25
s = 5.0
for seed in range(123, 1100, 100):
    for aug in [3, 7, 15, 22, 33]:
        config_path = f'config_files/event_pT_norm_SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'"{config_path}"')

        i += 1

"config_files/event_pT_norm_SB_5.0_pt_jet_aug_3_25x25_config_01.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_7_25x25_config_02.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_15_25x25_config_03.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_22_25x25_config_04.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_33_25x25_config_05.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_3_25x25_config_06.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_7_25x25_config_07.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_15_25x25_config_08.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_22_25x25_config_09.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_33_25x25_config_10.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_3_25x25_config_11.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_7_25x25_config_12.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_15_25x25_config_13.json"
"config_files/event_pT_norm_SB_5.0_pt_jet_aug_22_25x25_config_14.json"
"config_file

In [4]:
i = 1
res = 25
s = 3.0
for seed in range(123, 1100, 100):
    for aug in [33]:
        config_path = f'config_files/event_pT_norm_SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_01.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_02.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_03.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_04.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_05.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_06.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_07.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_08.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_09.json"
    "config_files/event_pT_norm_SB_3.0_jet_aug_33_25x25_config_10.json"


In [3]:
i = 1
res = 25
s = 4.0
for seed in range(123, 1100, 100):
    for aug in [3, 7, 15, 22, 33]:
        config_path = f'config_files/event_pT_norm_SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_SB_4.0_jet_aug_3_25x25_config_01.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_7_25x25_config_02.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_15_25x25_config_03.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_22_25x25_config_04.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_33_25x25_config_05.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_3_25x25_config_06.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_7_25x25_config_07.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_15_25x25_config_08.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_22_25x25_config_09.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_33_25x25_config_10.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_3_25x25_config_11.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_7_25x25_config_12.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_15_25x25_config_13.json"
    "config_files/event_pT_norm_SB_4.0_jet_aug_22_25x25_config_14.json

In [4]:
i = 1
res = 25
s = 4.0
for seed in range(123, 1100, 100):
    for aug in [3, 7, 15, 22, 33]:
        config_path = f'config_files/event_pT_norm_SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_3_25x25_config_01.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_7_25x25_config_02.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_15_25x25_config_03.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_22_25x25_config_04.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_33_25x25_config_05.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_3_25x25_config_06.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_7_25x25_config_07.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_15_25x25_config_08.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_22_25x25_config_09.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_33_25x25_config_10.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_3_25x25_config_11.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_7_25x25_config_12.json"
    "config_files/event_pT_norm_SB_4.0_pt_jet_aug_15_25x25_config_13.json"
    "config_files/event_pT_norm

## Event normalization with Dropout

In [3]:
i = 1
res = 25
for seed in range(123, 1100, 100):
    for s in range(11):
        config_path = f'config_files/event_pT_normalize_dp_origin_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}']
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1


    "config_files/event_pT_normalize_dp_origin_25x25_config_01.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_02.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_03.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_04.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_05.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_06.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_07.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_08.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_09.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_10.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_11.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_12.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_13.json"
    "config_files/event_pT_normalize_dp_origin_25x25_config_14.json"
    "config_files/event_pT_normali

In [3]:
i = 1
res = 25
aug = 10
for s in range(11):
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_normalize_dp_035_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_01.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_02.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_03.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_04.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_05.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_06.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_07.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_08.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_09.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_10.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_11.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_12.json"
    "config_files/event_pT_normalize_dp_035_jet_aug_10_25x25_config_13.json"

In [4]:
i = 1
res = 25
aug = 10
for s in range(11):
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_normalize_dp_035_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')


        i += 1

    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_01.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_02.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_03.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_04.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_05.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_06.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_07.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_08.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_09.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_10.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_11.json"
    "config_files/event_pT_normalize_dp_035_pt_jet_aug_10_25x25_config_12.json"
    "config_files/event_pT_normalize_dp_

In [3]:
i = 1
res = 25
s = 3.0
for aug in [3, 5, 7, 15, 22, 33]:
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_norm_dp_035_SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_01.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_02.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_03.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_04.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_05.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_06.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_07.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_08.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_09.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_3_25x25_config_10.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_5_25x25_config_11.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_5_25x25_config_12.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_5_25x25_co

In [5]:
i = 1
res = 25
s = 3.0
for aug in [3, 5, 7, 15, 22, 33]:
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_norm_dp_035_SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_01.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_02.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_03.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_04.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_05.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_06.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_07.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_08.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_09.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_3_25x25_config_10.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_5_25x25_config_11.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_pt_jet_aug_5_25x25_config_12.json"
    "config_files/event_pT_n

In [4]:
i = 1
res = 25
s = 3.0
for aug in [10]:
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_norm_dp_035_SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_01.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_02.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_03.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_04.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_05.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_06.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_07.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_08.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_09.json"
    "config_files/event_pT_norm_dp_035_SB_3.0_jet_aug_10_25x25_config_10.json"


In [3]:
i = 1
res = 25
s = 5.0
for aug in [3, 5, 7, 15, 22, 33]:
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_norm_dp_035_SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_01.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_02.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_03.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_04.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_05.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_06.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_07.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_08.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_09.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_3_25x25_config_10.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_5_25x25_config_11.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_5_25x25_config_12.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_jet_aug_5_25x25_co

In [4]:
i = 1
res = 25
s = 5.0
for aug in [3, 5, 7, 15, 22, 33]:
    for seed in range(123, 1100, 100):
        config_path = f'config_files/event_pT_norm_dp_035_SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_config_{i:02}.json'
        with open(os.path.join('CNN', config_path), 'w') as f:
            data = {}

            data['train_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}'] + [f'../Sample/HVmodel/data/pt_smearing_jet_rotation/{res}x{res}/{n:02}' for n in range(1, 1 + aug)]
            data['val_npy_paths'] = [f'../Sample/HVmodel/data/origin/{res}x{res}/val']
            data['seed'] = seed
            data['sensitivity'] = s
            data['luminosity'] = 139
            data['true_label_path'] = f'../Sample/HVmodel/data/split_val/{res}x{res}/mix_sample_test-4.npy'
            data['model_name'] = f'SB_{s:.1f}_pt_jet_aug_{aug}_{res}x{res}_event_pT_norm_dp_035'
            data['sample_type'] = f'Sensitivity: {s:.1f}, Resolution: {res}x{res}, pT smearing + Jet rotation: +{aug}, Seed: {seed}, event pT normalization'

            json.dump(data, f, indent=4)

        cmd = f'python train_CNN_pT_norm_dp.py {config_path}'
        # print(cmd)
        print(f'    "{config_path}"')

        i += 1

    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_01.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_02.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_03.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_04.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_05.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_06.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_07.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_08.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_09.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_3_25x25_config_10.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_5_25x25_config_11.json"
    "config_files/event_pT_norm_dp_035_SB_5.0_pt_jet_aug_5_25x25_config_12.json"
    "config_files/event_pT_n